<a href="https://colab.research.google.com/github/marjerie/bloomtree-cuda/blob/master/vertex_coloring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pwd

/content


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-19damwq8
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-19damwq8
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=6e878eac9a7aac65480e6851bbc8c2dccfa5e6e1f6cf64dbb8828f42a4fde6c3
  Stored in directory: /tmp/pip-ephem-wheel-cache-1lvoyzck/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [ ]:
%%cuda --name vc-fb.cu

#include <stdio.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <math.h>
#include <stdint.h>
#include <inttypes.h>

#define FORCE_INLINE

__device__ int Parent(int node);
__device__ int LeftChild(int node);
__device__ int RightChild(int node);
__device__ int Sibling(int node);
__device__ int calculate_lca(int u, int v);
__device__ void traversal(long int prev, long int lca, long int src, long int dest, int n, uint64_t *hash, bool *bit, int h, int m);
__device__ bool check_traversal_up(int prev, int lca, int src, int dest, int n, bool *mask);
__device__ bool check_traversal_down(int prev, int lca, int src, int dest, int n, bool *mask);
__device__ bool CheckBloom(long tid, uint64_t *hash_value, bool *bit, int h, int m);
__device__ void SetBloom(long tid, uint64_t *hash_value, bool *bit, int h, int m);

__device__ static inline FORCE_INLINE uint64_t rotl64 ( uint64_t x, int8_t r )
{
  return (x << r) | (x >> (64 - r));
}

#define ROTL64(x,y)	rotl64(x,y)
#define BIG_CONSTANT(x) (x##LLU)

#define getblock(p, i) (p[i])

__device__ static inline FORCE_INLINE uint64_t fmix64 ( uint64_t k )
{
  k ^= k >> 33;
  k *= BIG_CONSTANT(0xff51afd7ed558ccd);
  k ^= k >> 33;
  k *= BIG_CONSTANT(0xc4ceb9fe1a85ec53);
  k ^= k >> 33;

  return k;
}

__device__ void MurmurHash3_x64_128 ( int tid, char key[], const int len,
                           const uint32_t seed, void * out1 , void * out2)
{
  //const uint8_t * data = (const uint8_t*)key;
  //printf("tid = %d, %c, %c, %c, %c, %c, %c, %c, %c, %c\n",tid,key[0],key[1],key[2],key[3],key[4],key[5],key[6],key[7],key[8]);

  //if (key[0]== 48) {printf("h "); key[0] = 48+'\0';}

  uint8_t data[16];
  for (int i=0; i<len; i++)
    data[i] = (uint8_t) key[i];
  const int nblocks = len / 16;
  int i;

  uint64_t h1 = seed;
  uint64_t h2 = seed;

  uint64_t c1 = BIG_CONSTANT(0x87c37b91114253d5);
  uint64_t c2 = BIG_CONSTANT(0x4cf5ad432745937f);

  //----------
  // body

  //const uint64_t * blocks = (const uint64_t *)(data);

  uint64_t blocks[16];
  for (int i=0; i<len; i++)
    blocks[i] = (uint64_t) data[i];

  for(i = 0; i < nblocks; i++)
  {
    uint64_t k1 = blocks[i*2+0];
    uint64_t k2 = blocks[i*2+1];

    k1 *= c1; k1  = ROTL64(k1,31); k1 *= c2; h1 ^= k1;

    h1 = ROTL64(h1,27); h1 += h2; h1 = h1*5+0x52dce729;

    k2 *= c2; k2  = ROTL64(k2,33); k2 *= c1; h2 ^= k2;

    h2 = ROTL64(h2,31); h2 += h1; h2 = h2*5+0x38495ab5;
  }

  //----------
  // tail

  //const uint8_t * tail = (const uint8_t*)(data + nblocks*16);

  uint8_t tail[16];
  for (int i=0; i<len-nblocks*16; i++)
    tail[i] = (uint64_t) data[i+nblocks*16];

  uint64_t k1 = 0;
  uint64_t k2 = 0;

  switch(len & 15)
  {
  case 15: k2 ^= (uint64_t)(tail[14]) << 48;
  case 14: k2 ^= (uint64_t)(tail[13]) << 40;
  case 13: k2 ^= (uint64_t)(tail[12]) << 32;
  case 12: k2 ^= (uint64_t)(tail[11]) << 24;
  case 11: k2 ^= (uint64_t)(tail[10]) << 16;
  case 10: k2 ^= (uint64_t)(tail[ 9]) << 8;
  case  9: k2 ^= (uint64_t)(tail[ 8]) << 0;
           k2 *= c2; k2  = ROTL64(k2,33); k2 *= c1; h2 ^= k2;

  case  8: k1 ^= (uint64_t)(tail[ 7]) << 56;
  case  7: k1 ^= (uint64_t)(tail[ 6]) << 48;
  case  6: k1 ^= (uint64_t)(tail[ 5]) << 40;
  case  5: k1 ^= (uint64_t)(tail[ 4]) << 32;
  case  4: k1 ^= (uint64_t)(tail[ 3]) << 24;
  case  3: k1 ^= (uint64_t)(tail[ 2]) << 16;
  case  2: k1 ^= (uint64_t)(tail[ 1]) << 8;
  case  1: k1 ^= (uint64_t)(tail[ 0]) << 0;
           k1 *= c1; k1  = ROTL64(k1,31); k1 *= c2; h1 ^= k1;
  };

  //----------
  // finalization

  h1 ^= len; h2 ^= len;

  h1 += h2;
  h2 += h1;

  h1 = fmix64(h1);
  h2 = fmix64(h2);

  h1 += h2;
  h2 += h1;

  ((uint64_t*)out1)[0] = h1;
  ((uint64_t*)out2)[0] = h2;

}

__device__ inline uint64_t NthHash(uint8_t n, uint64_t hashA, uint64_t hashB, uint64_t filter_size) {
	//printf("%" PRIu64 " and %" PRIu64 " %u \n",hashA,hashB,((hashA + n * hashB) % filter_size));
	//printf("%u\n",n);
	//printf ("nth %u\n",((hashA + n * hashB) % filter_size));
	return ((hashA + n * hashB) % filter_size);
}


__global__ void init_mask(bool *mask, int n)
{
	int blockNum = blockIdx.z*(gridDim.x*gridDim.y)+blockIdx.y*gridDim.x+blockIdx.x;
	int threadNum = threadIdx.z*(blockDim.x*blockDim.y)+threadIdx.y*blockDim.x+threadIdx.x;
	int tid = blockNum*(blockDim.x*blockDim.y*blockDim.z)+threadNum;

	if (tid < n)
		mask[tid] = 0;
}

__global__ void init_bits(bool *array, int n)
{
	int tid = threadIdx.x + blockDim.x*blockIdx.x;
	if (tid < n){
		array[tid] = 0;
	}
}

__global__ void get_mask(int *u, int *v, bool *bit, int n, int e, long int ful_vertices,long int valsperloop,int ii,int h, int m, uint64_t *hash)
{
	int blockNum = blockIdx.z*(gridDim.x*gridDim.y)+blockIdx.y*gridDim.x+blockIdx.x;
	int threadNum = threadIdx.z*(blockDim.x*blockDim.y)+threadIdx.y*blockDim.x+threadIdx.x;
	int tid = blockNum*(blockDim.x*blockDim.y*blockDim.z)+threadNum;

	long int tid1 = ii*valsperloop + tid;

	if (tid1 < e){
		u[tid] = u[tid] + n - 1;
		v[tid] = v[tid] + n - 1;
		int src = u[tid];
		int dest = v[tid];

		if (!((u[tid] < ful_vertices && v[tid] < ful_vertices) || (u[tid] >= ful_vertices && v[tid] >= ful_vertices))) {
			if (u[tid] > v[tid]){
	 			int cur = Parent(u[tid]);
				SetBloom((cur*n+u[tid]-n+1) << 1,hash+2*tid*sizeof(uint64_t),bit,h,m);
				if (u[tid] == LeftChild(cur)){
					SetBloom((cur*n+v[tid]-n+1) << 1,hash+2*tid*sizeof(uint64_t),bit,h,m); 
				}
				else{
					SetBloom(((cur*n+v[tid]-n+1) << 1) + 1,hash+2*tid*sizeof(uint64_t),bit,h,m); 
				}
				u[tid] = cur;
			}	
		 	else{
	 			int cur = Parent(v[tid]);
				SetBloom((cur*n+v[tid]-n+1) << 1,hash+2*tid*sizeof(uint64_t),bit,h,m);
				if (v[tid] == LeftChild(cur)){
					SetBloom((cur*n+u[tid]-n+1) << 1,hash+2*tid*sizeof(uint64_t),bit,h,m); 
				}
				else{
					SetBloom(((cur*n+u[tid]-n+1) << 1) + 1,hash+2*tid*sizeof(uint64_t),bit,h,m); 
				}
				v[tid] = cur;
			}
		}
		
		__syncthreads();

		int lca = calculate_lca(u[tid], v[tid]);

		traversal(u[tid], lca, src, dest, n, hash+2*tid*sizeof(uint64_t),bit,h,m);
		traversal(v[tid], lca, dest, src, n, hash+2*tid*sizeof(uint64_t),bit,h,m);
	}
}

__global__ void SetBloom(bool *mask, uint64_t *hash_value, bool *bit, int h, int m, long int valsperloop, int ii)
{
	int blockNum = blockIdx.z*(gridDim.x*gridDim.y)+blockIdx.y*gridDim.x+blockIdx.x;
	int threadNum = threadIdx.z*(blockDim.x*blockDim.y)+threadIdx.y*blockDim.x+threadIdx.x;
	int tid = blockNum*(blockDim.x*blockDim.y*blockDim.z)+threadNum;

	if (mask[tid] == 1){
		long tid1 = valsperloop*ii + tid;
		int i =0;
		int count=0; 	
		long num = tid1;
		char str[10];
		do{	
			count++;
			num /= 10;
		}while(num != 0); 
		num = tid1;
		do{	
			str[count-i-1] = num%10 + '0';
			num/=10;
			i++;
		}while(num !=0);	
		str[i] = 48+'\0';
		uint64_t len1 = (uint64_t) count;
		size_t len = (size_t) len1;
		MurmurHash3_x64_128(tid1, str, len, 0, (hash_value)+tid*2*sizeof(uint64_t), (hash_value)+(tid*2+1)*sizeof(uint64_t));	
	
		for (int i=0; i<h; i++){
			bit[NthHash(i,*((hash_value)+tid*2*sizeof(uint64_t)),*((hash_value)+(tid*2+1)*sizeof(uint64_t)),m)] = 1;
		}
	}
}

__global__ void set_mask(bool *mask, uint64_t *hash_value, bool *bit, int h, int m, int n, long int valsperloop, int ii)
{
	int blockNum = blockIdx.z*(gridDim.x*gridDim.y)+blockIdx.y*gridDim.x+blockIdx.x;
	int threadNum = threadIdx.z*(blockDim.x*blockDim.y)+threadIdx.y*blockDim.x+threadIdx.x;
	int tid = blockNum*(blockDim.x*blockDim.y*blockDim.z)+threadNum;

	if ((valsperloop*ii + tid) < n){	
		long tid1 = valsperloop*ii + tid;
		mask[tid] = CheckBloom(tid1, (hash_value)+2*tid*sizeof(uint64_t), bit, h, m);
	}

}

__global__ void check_bloom(int *found, bool *bit, bool *mask, uint64_t *hash_value, int m, int h, int n)
{
	int val = blockIdx.x;
	int hash = threadIdx.x;
	uint64_t filter_size = (uint64_t) m;
	uint8_t hash_no = (uint8_t) hash;
	
	if (val < n && *found == 0){
		if (mask[val] == 1){
			 if (hash < h){
				if (bit[NthHash(hash_no,*(hash_value+2*val*sizeof(uint64_t)),*(hash_value+(2*val+1)*sizeof(uint64_t)), filter_size)] == 0){ 		
					atomicAdd(found, 1); //printf("here");
				}
			}
		}
	}
}


__global__ void get_neighbours(int u, bool *neighs, bool *mask, int n, long int ful_vertices)
{
	int blockNum = blockIdx.z*(gridDim.x*gridDim.y)+blockIdx.y*gridDim.x+blockIdx.x;
	int threadNum = threadIdx.z*(blockDim.x*blockDim.y)+threadIdx.y*blockDim.x+threadIdx.x;
	int tid = blockNum*(blockDim.x*blockDim.y*blockDim.z)+threadNum;

	long v=tid;

	*(neighs+(tid)*sizeof(bool)) = 0;

	if (tid < n && !(u==tid) ){

		u = u + n - 1;
		v = v + n - 1;
		long src = u;
		long dest = v;

		if (!((u < ful_vertices && v < ful_vertices) || (u >= ful_vertices && v >= ful_vertices))) {
			if (u > v){
	 			long cur = Parent(u);
				if ((mask[(cur*n+u-n+1) << 1] == 0)) return;
				u = cur;
			}	
		 	else{
	 			long cur = Parent(v);
				if (v == LeftChild(cur))
				{
					if ((mask[(cur*n+u-n+1) << 1] == 0)) 
						return;
				}
				else
				{
					if ((mask[((cur*n+u-n+1) << 1) + 1] == 0)) 
						return;
				} 
				v = cur;
			}
		}
	
		//__syncthreads();

		int lca = calculate_lca(u, v);
		if (check_traversal_up(u, lca, src, dest, n, mask)) 
		{
			if(check_traversal_down(v, lca, dest, src, n, mask))
			{
				//if ((src-n+1) == 6) printf("%d ",dest-n+1);
				*(neighs+(dest-n+1)*sizeof(bool)) = 1;
			}
		}
	}
}


__device__ int Parent(int node)
{
	return (((node + 1) >> 1) - 1); 
}

__device__ int LeftChild(int node)
{
	return (((node + 1) << 1) - 1); 
}

__device__ int RightChild(int node)
{
	return ((node + 1) << 1); 
}

__device__ int Sibling(int node)
{
	return (((node + 1) ^ 1) - 1); 
}

__device__ int calculate_lca(int u, int v)
{	
	int val1 = 0;
	int val2 = 0;	
	int i = 1;

	do{
		float pow_val = 1 << i;
		val1 = floor((u+1)/pow_val);
		val2 = floor((v+1)/pow_val);
		i++;
	} while(val1 != val2);
	return (val1 - 1);
}

__device__ void traversal(long int prev, long int lca, long int src, long int dest, int n, uint64_t *hash, bool *bit, int h, int m)
{
	int cur = Parent(prev);
	while (cur != lca){
		SetBloom((cur*n+src-n+1) << 1,hash,bit,h,m);
		if (prev == LeftChild(cur)){
			SetBloom((cur*n+dest-n+1) << 1,hash,bit,h,m); 
		}
		else{
			SetBloom(((cur*n+dest-n+1) << 1) + 1,hash,bit,h,m); 
		} 
		prev = cur;
		cur = Parent(cur);
	}
	SetBloom(((cur*n+src-n+1) << 1) + 1,hash,bit,h,m);	
}

__device__ bool check_traversal_up(int prev, int lca, int src, int dest, int n, bool *mask)  
{
	int cur = Parent(prev);
	while (cur != lca){
		if ((mask[(cur*n+src-n+1) << 1]) == 0)  return false;
		prev = cur;
		cur = Parent(cur);
	}
	if ((mask[((cur*n+src-n+1) << 1) + 1]) == 0)  return false;
	return true;	
}

__device__ bool check_traversal_down(int prev, int lca, int src, int dest, int n, bool *mask) 
{
	int cur = Parent(prev);
	while (cur != lca){
		if (prev == LeftChild(cur))
		{
			if ((mask[(cur*n+dest-n+1) << 1]) == 0)  
				return false;
		}
		else
		{
			if ((mask[((cur*n+dest-n+1) << 1) + 1]) == 0) 
				return false;
		}
		prev = cur;
		cur = Parent(cur);
	}
	return true;	
}

__device__ void SetBloom(long tid, uint64_t *hash_value, bool *bit, int h, int m)
{
	int i =0;
	int count=0; 	
	long num = tid;
	char str[11];
	do{	
		count++;
		num /= 10;
	}while(num != 0); 
	num = tid;
	do{	
		str[count-i-1] = num%10 + '0';
		num/=10;
		i++;
	}while(num !=0);	
	str[i] = 48+'\0';
	uint64_t len1 = (uint64_t) count;
	size_t len = (size_t) len1;
	MurmurHash3_x64_128(tid, str, len, 0, (hash_value), (hash_value)+sizeof(uint64_t));

	for (int i=0; i<h; i++){
		bit[NthHash(i,*(hash_value),*((hash_value)+sizeof(uint64_t)),m)] = 1;
	}
}

__device__ bool CheckBloom(long tid, uint64_t *hash_value, bool *bit, int h, int m)
{
	//printf("%ld ",tid);
	int i =0;
	int count=0; 	
	long num = tid;
	char str[10];
	do{	
		count++;
		num /= 10;
	}while(num != 0); 
	num = tid;
	do{	
		str[count-i-1] = num%10 + '0';
		num/=10;
		i++;
	}while(num !=0);	
	str[i] = 48+'\0';
	uint64_t len1 = (uint64_t) count;
	size_t len = (size_t) len1;
	MurmurHash3_x64_128(tid, str, len, 0, (hash_value), (hash_value)+sizeof(uint64_t));

	for (int i=0; i<h; i++){
		if (bit[NthHash(i,*(hash_value),*((hash_value)+sizeof(uint64_t)),m)] == 0){
			//printf("here ");
			//atomicAdd(no,1);
			return false;
		}
	}
	//atomicAdd(no,1);
	return true;
}

void InsertEdge(int num_vertices, int num_edges, int num_hashes, int num_bits, int *h_u, int *h_v, bool *h_bits)
{

	cudaError_t err = cudaSuccess;
	cudaEvent_t start1, stop1;
	cudaEventCreate(&start1);
	cudaEventCreate(&stop1);

	size_t size = num_edges * sizeof(int);
	//long int num_vals = 2692532344;

	printf("size is %zu\n",size);

	int num_ful_levels = floor( log2((double) (2*num_vertices - 1)));
	long int ful_vertices = pow((int) 2,(int) num_ful_levels) - 1;

	size_t size_bits = num_bits * sizeof(bool);
	bool *d_bits = NULL;
        err=cudaMalloc((void **)&d_bits, size_bits);
	cudaDeviceSynchronize();
	if (err != cudaSuccess)
	{
	    fprintf(stderr, "Failed to allocate vector bits (error code %s)!\n", cudaGetErrorString(err));
	    exit(EXIT_FAILURE);
	}

	cudaMemset(d_bits, 0, size_bits);

	int *d_u = NULL, *d_v = NULL; 
	err=cudaMalloc((void **)&d_u, size);
	cudaDeviceSynchronize();
	if (err != cudaSuccess)
	{
	    fprintf(stderr, "Failed to allocate vector u (error code %s)!\n", cudaGetErrorString(err));
	    exit(EXIT_FAILURE);
	}
	err=cudaMalloc((void **)&d_v, size);
	cudaDeviceSynchronize();
	if (err != cudaSuccess)
	{
	    fprintf(stderr, "Failed to allocate vector v (error code %s)!\n", cudaGetErrorString(err));
	    exit(EXIT_FAILURE);
	}
	err=cudaMemcpy(d_u, h_u, size, cudaMemcpyHostToDevice);
	cudaDeviceSynchronize();
	if (err != cudaSuccess)
	{
	    fprintf(stderr, "Failed to copy u (error code %s)!\n", cudaGetErrorString(err));
	    exit(EXIT_FAILURE);
	}
	err=cudaMemcpy(d_v, h_v, size, cudaMemcpyHostToDevice);
	cudaDeviceSynchronize();
	if (err != cudaSuccess)
	{
	    fprintf(stderr, "Failed to copy v (error code %s)!\n", cudaGetErrorString(err));
	    exit(EXIT_FAILURE);
	}

	clock_t ti;

	cudaEventRecord(start1);
	ti=clock();

	dim3 tpb1(32,16,1);
   	dim3 bpg1(8,2,1);

	long int valsperloop = 1 << 13;

	int N = ceil(num_edges/valsperloop)+1;

	uint64_t *d_hash_value = NULL;
	size_t size_hash = 2*num_edges*sizeof(uint64_t);
	cudaMalloc((void **)&d_hash_value, size_hash);

	//printf("N is %d\n",N);

	for (int i=0; i<N; i++){
		get_mask<<<bpg1,tpb1>>>(d_u+i*valsperloop, d_v+i*valsperloop, d_bits, num_vertices, num_edges, ful_vertices, valsperloop, i, num_hashes, num_bits, d_hash_value);

		/*cudaDeviceSynchronize();
	
		err = cudaGetLastError();

		if (err != cudaSuccess)
		{
		    fprintf(stderr, "Failed to launch get_mask kernel %d (error code %s)!\n",i, cudaGetErrorString(err));
		    exit(EXIT_FAILURE);
		}*/
	}

	cudaMemcpy(h_bits, d_bits, size_bits, cudaMemcpyDeviceToHost);

	ti=clock() -ti;
	cudaEventRecord(stop1);

	cudaEventSynchronize(stop1);
	float milliseconds = 0;
	cudaEventElapsedTime(&milliseconds, start1, stop1);

	printf("%.5f\n", float(ti) / CLOCKS_PER_SEC);

	printf("time taken is %.5f\n", milliseconds);

	int value=0;

	for (long i =0; i<num_bits; i++){
		if (h_bits[i] == 1) value++;
	}

	printf("value is %d\n",value);

	cudaFree(d_u);
	cudaFree(d_v);
	cudaFree(d_hash_value);
	cudaFree(d_bits);
}

int main ()
{

	cudaError_t err = cudaSuccess;
	const int INF = 1e8;
	int num_vertices, num_edges, num_hashes, num_bits;
	scanf("%d",&num_vertices);
	scanf("%d",&num_edges);
	scanf("%d",&num_bits);
	scanf("%d",&num_hashes);

	cudaEvent_t start, stop;
	cudaEventCreate(&start);
	cudaEventCreate(&stop);

	size_t size = num_edges * sizeof(int);
	int num_vals = 2*num_vertices*(num_vertices-1);

	int *h_u = (int *)malloc(size);
	int *h_v = (int *)malloc(size);
	
	for (int i =0; i<num_edges; i++)
	{
		scanf("%d",&h_u[i]);
		scanf("%d",&h_v[i]);
	}

	size_t size_bits = num_bits * sizeof(bool);
	bool *h_bits = (bool *)malloc(size_bits);

	InsertEdge(num_vertices, num_edges, num_hashes, num_bits, h_u, h_v, h_bits);

	uint64_t *d_hash_value = NULL;
	size_t size_hash = 2*num_vals*sizeof(uint64_t);
	cudaMalloc((void **)&d_hash_value, size_hash);

	size_t size_mask = num_vals * sizeof(bool);
	bool *h_mask = (bool *)malloc(size_mask);
	bool *d_mask = NULL;
    	cudaMalloc((void **)&d_mask, size_mask);

	bool *d_bits = NULL;
    	cudaMalloc((void **)&d_bits, size_bits);
	cudaMemcpy(d_bits, h_bits, size_bits, cudaMemcpyHostToDevice);

	bool *d_neighs = NULL;
	size_t size_neighs = (num_vertices)*sizeof(bool);
	cudaMalloc((void **)&d_neighs, size_neighs);
	bool *h_neighs = (bool *)malloc(size_neighs);
	
	int num_ful_levels = floor( log2((double) (2*num_vertices - 1)));
	long int ful_vertices = pow((int) 2,(int) num_ful_levels) - 1;

	size_t size_c = num_vertices * sizeof(int);
	int *color = (int *)malloc(size_c);
	size_t size_n = num_vertices * sizeof(bool);
	bool *is_neighbour_color = (bool *)malloc(size_n);

	dim3 tpb2(32,32,1);
    	dim3 bpg2(2,2,1);

	//dim3 tpb3(32,32,1);
    	//dim3 bpg3(16,16,1);

	//long int valsperloop = 1 << 18;

	dim3 tpb3(32,32,1);
    	dim3 bpg3(4,4,1);
	long int valsperloop = 1 << 14;

	int N = ceil(num_vals/valsperloop)+1;

	//printf("N is %d\n",N);

	clock_t ti;

	cudaEventRecord(start);

	ti=clock();

	for (int c = 1; c <= num_vertices ; ++c) {
		is_neighbour_color[c] = false;
	}

	for (int v = 0; v < num_vertices; ++v) {
		color[v] = -1;
	}

	color[0] = 1;
	int num_colors_used = 1;

	for (int i=0; i<N; i++){
		set_mask<<<bpg3,tpb3>>>(d_mask+i*valsperloop,d_hash_value+i*2*valsperloop,d_bits,num_hashes,num_bits,num_vals,valsperloop,i);

        /*cudaDeviceSynchronize();
	
		err = cudaGetLastError();

		if (err != cudaSuccess)
		{
		    fprintf(stderr, "Failed to launch set_mask kernel %d (error code %s)!\n",i, cudaGetErrorString(err));
		    exit(EXIT_FAILURE);
		}*/
	}

	for (int v = 1; v < num_vertices; ++v)
	{
		get_neighbours<<<bpg2,tpb2>>>(v, d_neighs, d_mask, num_vertices, ful_vertices);
		cudaMemcpy(h_neighs, d_neighs, size_neighs, cudaMemcpyDeviceToHost);
		
		for (int i = 0; i < num_vertices; ++i) {
			if (h_neighs[i] == 1){
				if (color[i] != -1)
					is_neighbour_color[color[i]] = true;
			}
		}

		// Finding first unassigned colour
		int c;
		for (c = 1; c <= num_vertices; ++c) {
			if (is_neighbour_color[c] == false) break;
		}
		color[v] = c;

		for (int i = 0; i < num_vertices; ++i) {
			if (h_neighs[i] == 1){
				if (color[i] != -1)
					is_neighbour_color[color[i]] = false;
			}
		}

		if(c > num_colors_used) num_colors_used = c;
	}

	ti=clock()-ti;

	cudaEventRecord(stop);

	cudaEventSynchronize(stop);
	float milliseconds = 0;
	cudaEventElapsedTime(&milliseconds, start, stop);

	printf("number of colours used: %d\n",num_colors_used);

	printf("%.5f\n", float(ti) / CLOCKS_PER_SEC);

	printf("time taken is %.5f\n", milliseconds);

	cudaFree(d_neighs);
	cudaFree(d_hash_value);
	cudaFree(d_bits);
	cudaFree(d_mask);

	free(h_u);
	free(h_v);
	free(h_bits);
	
	cudaDeviceReset();
	return 0;
}

'File written in /content/src/vc-fb.cu'

In [ ]:
!nvidia-smi

Thu Oct 22 10:49:07 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    18W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!nvcc /content/src/vc-fb.cu -o vc-fb

/content/src/vc-fb.cu(595): warning: variable "err" was declared but never referenced

/content/src/vc-fb.cu(596): warning: variable "INF" was declared but never referenced



In [ ]:
!./vc-fb < facebook_combined.txt

size is 352936
0.01149
time taken is 11.51469
value is 1157630
number of colours used: 162
0.36847
time taken is 369.10580


In [ ]:
!nvprof --unified-memory-profiling off ./vc-fb < facebook_combined.txt

==534== NVPROF is profiling process 534, command: ./vc-fb
size is 352936
0.01199
time taken is 12.00195
value is 1157630
number of colours used: 162
0.38417
time taken is 385.12125
==534== Profiling application: ./vc-fb
==534== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   46.98%  91.571ms      4038  22.677us  11.359us  28.799us  get_neighbours(int, bool*, bool*, int, long)
                   43.30%  84.405ms      1991  42.393us  36.447us  44.542us  set_mask(bool*, unsigned long*, bool*, int, int, int, long, int)
                    5.58%  10.869ms        11  988.10us  867.59us  1.0572ms  get_mask(int*, int*, bool*, int, int, long, long, int, int, int, unsigned long*)
                    4.05%  7.8964ms      4039  1.9550us  1.7910us  97.629us  [CUDA memcpy DtoH]
                    0.09%  180.83us         3  60.275us  33.855us  112.28us  [CUDA memcpy HtoD]
                    0.00%  1.9200us         1  1.9200us  1

In [ ]:
%%cuda --name vc-as.cu

#include <stdio.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <math.h>
#include <stdint.h>
#include <inttypes.h>

#define FORCE_INLINE

__device__ int Parent(int node);
__device__ int LeftChild(int node);
__device__ int RightChild(int node);
__device__ int Sibling(int node);
__device__ int calculate_lca(int u, int v);
__device__ void traversal(long int prev, long int lca, long int src, long int dest, int n, uint64_t *hash, bool *bit, int h, int m);
__device__ bool check_traversal_up(int prev, int lca, int src, int dest, int n, bool *mask);
__device__ bool check_traversal_down(int prev, int lca, int src, int dest, int n, bool *mask);
__device__ bool CheckBloom(long tid, uint64_t *hash_value, bool *bit, int h, int m);
__device__ void SetBloom(long tid, uint64_t *hash_value, bool *bit, int h, int m);

__device__ static inline FORCE_INLINE uint64_t rotl64 ( uint64_t x, int8_t r )
{
  return (x << r) | (x >> (64 - r));
}

#define ROTL64(x,y)	rotl64(x,y)
#define BIG_CONSTANT(x) (x##LLU)

#define getblock(p, i) (p[i])

__device__ static inline FORCE_INLINE uint64_t fmix64 ( uint64_t k )
{
  k ^= k >> 33;
  k *= BIG_CONSTANT(0xff51afd7ed558ccd);
  k ^= k >> 33;
  k *= BIG_CONSTANT(0xc4ceb9fe1a85ec53);
  k ^= k >> 33;

  return k;
}

__device__ void MurmurHash3_x64_128 ( int tid, char key[], const int len,
                           const uint32_t seed, void * out1 , void * out2)
{
  //const uint8_t * data = (const uint8_t*)key;
  //printf("tid = %d, %c, %c, %c, %c, %c, %c, %c, %c, %c, %c, %c\n",tid,key[0],key[1],key[2],key[3],key[4],key[5],key[6],key[7],key[8],key[9],key[10]);

  //if (key[0]== 48) {printf("h "); key[0] = 48+'\0';}

  uint8_t data[16];
  for (int i=0; i<len; i++)
    data[i] = (uint8_t) key[i];
  const int nblocks = len / 16;
  int i;

  uint64_t h1 = seed;
  uint64_t h2 = seed;

  uint64_t c1 = BIG_CONSTANT(0x87c37b91114253d5);
  uint64_t c2 = BIG_CONSTANT(0x4cf5ad432745937f);

  //----------
  // body

  //const uint64_t * blocks = (const uint64_t *)(data);

  uint64_t blocks[16];
  for (int i=0; i<len; i++)
    blocks[i] = (uint64_t) data[i];

  for(i = 0; i < nblocks; i++)
  {
    uint64_t k1 = blocks[i*2+0];
    uint64_t k2 = blocks[i*2+1];

    k1 *= c1; k1  = ROTL64(k1,31); k1 *= c2; h1 ^= k1;

    h1 = ROTL64(h1,27); h1 += h2; h1 = h1*5+0x52dce729;

    k2 *= c2; k2  = ROTL64(k2,33); k2 *= c1; h2 ^= k2;

    h2 = ROTL64(h2,31); h2 += h1; h2 = h2*5+0x38495ab5;
  }

  //----------
  // tail

  //const uint8_t * tail = (const uint8_t*)(data + nblocks*16);

  uint8_t tail[16];
  for (int i=0; i<len-nblocks*16; i++)
    tail[i] = (uint64_t) data[i+nblocks*16];

  uint64_t k1 = 0;
  uint64_t k2 = 0;

  switch(len & 15)
  {
  case 15: k2 ^= (uint64_t)(tail[14]) << 48;
  case 14: k2 ^= (uint64_t)(tail[13]) << 40;
  case 13: k2 ^= (uint64_t)(tail[12]) << 32;
  case 12: k2 ^= (uint64_t)(tail[11]) << 24;
  case 11: k2 ^= (uint64_t)(tail[10]) << 16;
  case 10: k2 ^= (uint64_t)(tail[ 9]) << 8;
  case  9: k2 ^= (uint64_t)(tail[ 8]) << 0;
           k2 *= c2; k2  = ROTL64(k2,33); k2 *= c1; h2 ^= k2;

  case  8: k1 ^= (uint64_t)(tail[ 7]) << 56;
  case  7: k1 ^= (uint64_t)(tail[ 6]) << 48;
  case  6: k1 ^= (uint64_t)(tail[ 5]) << 40;
  case  5: k1 ^= (uint64_t)(tail[ 4]) << 32;
  case  4: k1 ^= (uint64_t)(tail[ 3]) << 24;
  case  3: k1 ^= (uint64_t)(tail[ 2]) << 16;
  case  2: k1 ^= (uint64_t)(tail[ 1]) << 8;
  case  1: k1 ^= (uint64_t)(tail[ 0]) << 0;
           k1 *= c1; k1  = ROTL64(k1,31); k1 *= c2; h1 ^= k1;
  };

  //----------
  // finalization

  h1 ^= len; h2 ^= len;

  h1 += h2;
  h2 += h1;

  h1 = fmix64(h1);
  h2 = fmix64(h2);

  h1 += h2;
  h2 += h1;

  ((uint64_t*)out1)[0] = h1;
  ((uint64_t*)out2)[0] = h2;

}

__device__ inline uint64_t NthHash(uint8_t n, uint64_t hashA, uint64_t hashB, uint64_t filter_size) {
	//printf("%" PRIu64 " and %" PRIu64 " %u \n",hashA,hashB,((hashA + n * hashB) % filter_size));
	//printf("%u\n",n);
	//printf ("nth %u\n",((hashA + n * hashB) % filter_size));
	return ((hashA + n * hashB) % filter_size);
}

__global__ void get_mask(int *u, int *v, bool *bit, int n, int e, long int ful_vertices,long int valsperloop,int ii,int h, int m, uint64_t *hash)
{
	int blockNum = blockIdx.z*(gridDim.x*gridDim.y)+blockIdx.y*gridDim.x+blockIdx.x;
	int threadNum = threadIdx.z*(blockDim.x*blockDim.y)+threadIdx.y*blockDim.x+threadIdx.x;
	int tid = blockNum*(blockDim.x*blockDim.y*blockDim.z)+threadNum;

	long int tid1 = ii*valsperloop + tid;

	if (tid1 < e){
		u[tid] = u[tid] + n - 1;
		v[tid] = v[tid] + n - 1;
		int src = u[tid];
		int dest = v[tid];

		if (!((u[tid] < ful_vertices && v[tid] < ful_vertices) || (u[tid] >= ful_vertices && v[tid] >= ful_vertices))) {
			if (u[tid] > v[tid]){
	 			int cur = Parent(u[tid]);
				SetBloom((cur*n+u[tid]-n+1) << 1,hash+2*tid*sizeof(uint64_t),bit,h,m);
				if (u[tid] == LeftChild(cur)){
					SetBloom((cur*n+v[tid]-n+1) << 1,hash+2*tid*sizeof(uint64_t),bit,h,m); 
				}
				else{
					SetBloom(((cur*n+v[tid]-n+1) << 1) + 1,hash+2*tid*sizeof(uint64_t),bit,h,m); 
				}
				u[tid] = cur;
			}	
		 	else{
	 			int cur = Parent(v[tid]);
				SetBloom((cur*n+v[tid]-n+1) << 1,hash+2*tid*sizeof(uint64_t),bit,h,m);
				if (v[tid] == LeftChild(cur)){
					SetBloom((cur*n+u[tid]-n+1) << 1,hash+2*tid*sizeof(uint64_t),bit,h,m); 
				}
				else{
					SetBloom(((cur*n+u[tid]-n+1) << 1) + 1,hash+2*tid*sizeof(uint64_t),bit,h,m); 
				}
				v[tid] = cur;
			}
		}
		
		__syncthreads();

		int lca = calculate_lca(u[tid], v[tid]);

		traversal(u[tid], lca, src, dest, n, hash+2*tid*sizeof(uint64_t),bit,h,m);
		traversal(v[tid], lca, dest, src, n, hash+2*tid*sizeof(uint64_t),bit,h,m);
	}
}

__global__ void set_mask(bool *mask, uint64_t *hash_value, bool *bit, int h, int m, int n, long int valsperloop, int ii)
{
	int blockNum = blockIdx.z*(gridDim.x*gridDim.y)+blockIdx.y*gridDim.x+blockIdx.x;
	int threadNum = threadIdx.z*(blockDim.x*blockDim.y)+threadIdx.y*blockDim.x+threadIdx.x;
	int tid = blockNum*(blockDim.x*blockDim.y*blockDim.z)+threadNum;

	if ((valsperloop*ii + tid) < n){	
		long tid1 = valsperloop*ii + tid;
		mask[tid] = CheckBloom(tid1, (hash_value)+2*tid*sizeof(uint64_t), bit, h, m);
	}
}

__global__ void check_bloom(int *found, bool *bit, bool *mask, uint64_t *hash_value, int m, int h, int n)
{
	int val = blockIdx.x;
	int hash = threadIdx.x;
	uint64_t filter_size = (uint64_t) m;
	uint8_t hash_no = (uint8_t) hash;
	
	if (val < n && *found == 0){
		if (mask[val] == 1){
			 if (hash < h){
				if (bit[NthHash(hash_no,*(hash_value+2*val*sizeof(uint64_t)),*(hash_value+(2*val+1)*sizeof(uint64_t)), filter_size)] == 0){ 		
					atomicAdd(found, 1); //printf("here");
				}
			}
		}
	}
}


__global__ void get_neighbours(int u, bool *neighs, bool *mask, int n, long int ful_vertices)
{
	int blockNum = blockIdx.z*(gridDim.x*gridDim.y)+blockIdx.y*gridDim.x+blockIdx.x;
	int threadNum = threadIdx.z*(blockDim.x*blockDim.y)+threadIdx.y*blockDim.x+threadIdx.x;
	int tid = blockNum*(blockDim.x*blockDim.y*blockDim.z)+threadNum;

	long v=tid;

	*(neighs+(tid)*sizeof(bool)) = 0;

	if (tid < n && !(u==tid) ){

		u = u + n - 1;
		v = v + n - 1;
		long src = u;
		long dest = v;

		if (!((u < ful_vertices && v < ful_vertices) || (u >= ful_vertices && v >= ful_vertices))) {
			if (u > v){
	 			long cur = Parent(u);
				if ((mask[(cur*n+u-n+1) << 1] == 0)) return;
				u = cur;
			}	
		 	else{
	 			long cur = Parent(v);
				if (v == LeftChild(cur))
				{
					if ((mask[(cur*n+u-n+1) << 1] == 0)) 
						return;
				}
				else
				{
					if ((mask[((cur*n+u-n+1) << 1) + 1] == 0)) 
						return;
				} 
				v = cur;
			}
		}
	
		//__syncthreads();

		int lca = calculate_lca(u, v);
		if (check_traversal_up(u, lca, src, dest, n, mask)) 
		{
			if(check_traversal_down(v, lca, dest, src, n, mask))
			{
				*(neighs+(dest-n+1)*sizeof(bool)) = 1;
			}
		}
	}
}

__global__ void print_hash(uint64_t *hash_value, int n)
{
	int blockNum = blockIdx.z*(gridDim.x*gridDim.y)+blockIdx.y*gridDim.x+blockIdx.x;
	int threadNum = threadIdx.z*(blockDim.x*blockDim.y)+threadIdx.y*blockDim.x+threadIdx.x;
	int tid = blockNum*(blockDim.x*blockDim.y*blockDim.z)+threadNum;

	if (tid < 4*n*(n-1)){
		//printf("%u\n",*(hash_value+tid*sizeof(uint8_t)));
		printf("%" PRIu64 " %d\n",*(hash_value+tid*sizeof(uint64_t)),tid);
	}	
}

__device__ int Parent(int node)
{
	return (((node + 1) >> 1) - 1); 
}

__device__ int LeftChild(int node)
{
	return (((node + 1) << 1) - 1); 
}

__device__ int RightChild(int node)
{
	return ((node + 1) << 1); 
}

__device__ int Sibling(int node)
{
	return (((node + 1) ^ 1) - 1); 
}

__device__ int calculate_lca(int u, int v)
{	
	int val1 = 0;
	int val2 = 0;	
	int i = 1;

	do{
		float pow_val = 1 << i;
		val1 = floor((u+1)/pow_val);
		val2 = floor((v+1)/pow_val);
		i++;
	} while(val1 != val2);
	return (val1 - 1);
}

__device__ void traversal(long int prev, long int lca, long int src, long int dest, int n, uint64_t *hash, bool *bit, int h, int m)
{
	int cur = Parent(prev);
	while (cur != lca){
		SetBloom((cur*n+src-n+1) << 1,hash,bit,h,m);
		if (prev == LeftChild(cur)){
			SetBloom((cur*n+dest-n+1) << 1,hash,bit,h,m); 
		}
		else{
			SetBloom(((cur*n+dest-n+1) << 1) + 1,hash,bit,h,m); 
		} 
		prev = cur;
		cur = Parent(cur);
	}
	SetBloom(((cur*n+src-n+1) << 1) + 1,hash,bit,h,m);	
}

__device__ bool check_traversal_up(int prev, int lca, int src, int dest, int n, bool *mask)  
{
	int cur = Parent(prev);
	while (cur != lca){
		if ((mask[(cur*n+src-n+1) << 1]) == 0)  return false;
		prev = cur;
		cur = Parent(cur);
	}
	if ((mask[((cur*n+src-n+1) << 1) + 1]) == 0)  return false;
	return true;	
}

__device__ bool check_traversal_down(int prev, int lca, int src, int dest, int n, bool *mask) 
{
	int cur = Parent(prev);
	while (cur != lca){
		if (prev == LeftChild(cur))
		{
			if ((mask[(cur*n+dest-n+1) << 1]) == 0)  
				return false;
		}
		else
		{
			if ((mask[((cur*n+dest-n+1) << 1) + 1]) == 0) 
				return false;
		}
		prev = cur;
		cur = Parent(cur);
	}
	return true;	
}


__device__ void SetBloom(long tid, uint64_t *hash_value, bool *bit, int h, int m)
{
	int i =0;
	int count=0; 	
	long num = tid;
	char str[11];
	do{	
		count++;
		num /= 10;
	}while(num != 0); 
	num = tid;
	do{	
		str[count-i-1] = num%10 + '0';
		num/=10;
		i++;
	}while(num !=0);	
	str[i] = 48+'\0';
	uint64_t len1 = (uint64_t) count;
	size_t len = (size_t) len1;
	MurmurHash3_x64_128(tid, str, len, 0, (hash_value), (hash_value)+sizeof(uint64_t));

	for (int i=0; i<h; i++){
		bit[NthHash(i,*(hash_value),*((hash_value)+sizeof(uint64_t)),m)] = 1;
	}
}

__device__ bool CheckBloom(long tid, uint64_t *hash_value, bool *bit, int h, int m)
{
	int i =0;
	int count=0; 	
	long num = tid;
	char str[10];
	do{	
		count++;
		num /= 10;
	}while(num != 0); 
	num = tid;
	do{	
		str[count-i-1] = num%10 + '0';
		num/=10;
		i++;
	}while(num !=0);	
	str[i] = 48+'\0';
	uint64_t len1 = (uint64_t) count;
	size_t len = (size_t) len1;
	MurmurHash3_x64_128(tid, str, len, 0, (hash_value), (hash_value)+sizeof(uint64_t));

	for (int i=0; i<h; i++){
		if (bit[NthHash(i,*(hash_value),*((hash_value)+sizeof(uint64_t)),m)] == 0){
			return false;
		}
	}
	return true;
}

void InsertEdge(int num_vertices, int num_edges, int num_hashes, int num_bits, int *h_u, int *h_v, bool *h_bits)
{

	cudaError_t err = cudaSuccess;
	cudaEvent_t start1, stop1;
	cudaEventCreate(&start1);
	cudaEventCreate(&stop1);

	size_t size = num_edges * sizeof(int);
	//long int num_vals = 2692532344;

	int *d_u = NULL, *d_v = NULL; 
    err=cudaMalloc((void **)&d_u, size);
	cudaDeviceSynchronize();
	if (err != cudaSuccess)
	{
	    fprintf(stderr, "Failed to allocate vector u (error code %s)!\n", cudaGetErrorString(err));
	    exit(EXIT_FAILURE);
	}
	err=cudaMalloc((void **)&d_v, size);
	cudaDeviceSynchronize();
	if (err != cudaSuccess)
	{
	    fprintf(stderr, "Failed to allocate vector v (error code %s)!\n", cudaGetErrorString(err));
	    exit(EXIT_FAILURE);
	}
	err=cudaMemcpy(d_u, h_u, size, cudaMemcpyHostToDevice);
	cudaDeviceSynchronize();
	if (err != cudaSuccess)
	{
	    fprintf(stderr, "Failed to copy u (error code %s)!\n", cudaGetErrorString(err));
	    exit(EXIT_FAILURE);
	}
	err=cudaMemcpy(d_v, h_v, size, cudaMemcpyHostToDevice);
	cudaDeviceSynchronize();
	if (err != cudaSuccess)
	{
	    fprintf(stderr, "Failed to copy v (error code %s)!\n", cudaGetErrorString(err));
	    exit(EXIT_FAILURE);
	}

	int num_ful_levels = floor( log2((double) (2*num_vertices - 1)));
	long int ful_vertices = pow((int) 2,(int) num_ful_levels) - 1;

	//dim3 tpb(32,32,1);
	//dim3 bpg(ceil(num_vertices/32),ceil((num_vertices-1)/32),2);

	//init_mask<<<bpg,tpb>>>(d_mask,num_vertices);

	size_t size_bits = num_bits * sizeof(bool);
	//bool *h_bits = (bool *)malloc(size_bits);
	bool *d_bits = NULL;
    err=cudaMalloc((void **)&d_bits, size_bits);
	cudaDeviceSynchronize();
	if (err != cudaSuccess)
	{
	    fprintf(stderr, "Failed to allocate vector bits (error code %s)!\n", cudaGetErrorString(err));
	    exit(EXIT_FAILURE);
	}

	//init_mask<<<2,num_bits/2>>>(d_bits,num_bits);
	cudaMemset(d_bits, 0, size_bits);

	//int M = num_edges/4096;
	//printf("M is %d\n",M);

	//dim3 tpb1(8,8,1);
    //dim3 bpg1(4,4,1);
	clock_t ti;

	cudaEventRecord(start1);
	ti=clock();

	dim3 tpb1(32,32,1);
    dim3 bpg1(4,1,1);

	long int valsperloop = 1 << 12;

	int N = ceil(num_edges/valsperloop)+1;

	uint64_t *d_hash_value = NULL;
	size_t size_hash = 2*valsperloop*sizeof(uint64_t);
	cudaMalloc((void **)&d_hash_value, size_hash);

	//printf("N is %d\n",N);

	for (int i=0; i<N; i++){
		get_mask<<<bpg1,tpb1>>>(d_u+i*valsperloop, d_v+i*valsperloop, d_bits, num_vertices, num_edges, ful_vertices, valsperloop, i, num_hashes, num_bits, d_hash_value);

		/*cudaDeviceSynchronize();
	
		err = cudaGetLastError();

		if (err != cudaSuccess)
		{
		    fprintf(stderr, "Failed to launch get_mask kernel %d (error code %s)!\n",i, cudaGetErrorString(err));
		    exit(EXIT_FAILURE);
		}*/
	}

	cudaMemcpy(h_bits, d_bits, size_bits, cudaMemcpyDeviceToHost);

	ti=clock() -ti;
	cudaEventRecord(stop1);

	cudaEventSynchronize(stop1);
	float milliseconds = 0;
	cudaEventElapsedTime(&milliseconds, start1, stop1);

	printf("%.5f\n", float(ti) / CLOCKS_PER_SEC);

	printf("time taken is %.5f\n", milliseconds);

	int value=0;

	for (long i =0; i<num_bits; i++){
		if (h_bits[i] == 1) value++;
	}

	printf("value is %d\n",value);

	cudaFree(d_u);
	cudaFree(d_v);
	cudaFree(d_hash_value);
	cudaFree(d_bits);
}


int main ()
{

	cudaError_t err = cudaSuccess;
	const int INF = 1e8;
	int num_vertices, num_edges, num_hashes, num_bits;
	scanf("%d",&num_vertices);
	scanf("%d",&num_edges);
	scanf("%d",&num_bits);
	scanf("%d",&num_hashes);

	cudaEvent_t start, stop;
	cudaEventCreate(&start);
	cudaEventCreate(&stop);

	size_t size = num_edges * sizeof(int);
	int num_vals = 2*num_vertices*(num_vertices-1);

	int *h_u = (int *)malloc(size);
	int *h_v = (int *)malloc(size);
	
	for (int i =0; i<num_edges; i++)
	{
		scanf("%d",&h_u[i]);
		scanf("%d",&h_v[i]);
	}

	size_t size_bits = num_bits * sizeof(bool);
	bool *h_bits = (bool *)malloc(size_bits);

	InsertEdge(num_vertices, num_edges, num_hashes, num_bits, h_u, h_v, h_bits);

	uint64_t *d_hash_value = NULL;
	size_t size_hash = 2*num_vals*sizeof(uint64_t);
	cudaMalloc((void **)&d_hash_value, size_hash);

	size_t size_mask = num_vals * sizeof(bool);
	bool *h_mask = (bool *)malloc(size_mask);
	bool *d_mask = NULL;
    cudaMalloc((void **)&d_mask, size_mask);

	bool *d_bits = NULL;
    cudaMalloc((void **)&d_bits, size_bits);
	cudaMemcpy(d_bits, h_bits, size_bits, cudaMemcpyHostToDevice);

	bool *d_neighs = NULL;
	size_t size_neighs = (num_vertices)*sizeof(bool);
	cudaMalloc((void **)&d_neighs, size_neighs);
	bool *h_neighs = (bool *)malloc(size_neighs);
	
	int num_ful_levels = floor( log2((double) (2*num_vertices - 1)));
	long int ful_vertices = pow((int) 2,(int) num_ful_levels) - 1;

	size_t size_c = num_vertices * sizeof(int);
	int *color = (int *)malloc(size_c);
	size_t size_n = num_vertices * sizeof(bool);
	bool *is_neighbour_color = (bool *)malloc(size_n);

	dim3 tpb2(32,32,1);
    	dim3 bpg2(4,2,1);

	dim3 tpb3(32,32,1);
    	dim3 bpg3(32,16,1);
	long int valsperloop = 1 << 19;

	int N = ceil(num_vals/valsperloop)+1;

	cudaEventRecord(start);

	for (int c = 1; c <= num_vertices ; ++c) {
		is_neighbour_color[c] = false;
	}

	for (int v = 0; v < num_vertices; ++v) {
		color[v] = -1;
	}

	color[0] = 1;
	int num_colors_used = 1;

	//printf("N is %d\n",N);

	for (int i=0; i<N; i++){
		set_mask<<<bpg3,tpb3>>>(d_mask+i*valsperloop,d_hash_value+i*2*valsperloop,d_bits,num_hashes,num_bits,num_vals,valsperloop,i);

        /*cudaDeviceSynchronize();
	
		err = cudaGetLastError();

		if (err != cudaSuccess)
		{
		    fprintf(stderr, "Failed to launch set_mask kernel %d (error code %s)!\n",i, cudaGetErrorString(err));
		    exit(EXIT_FAILURE);
		}*/
	}

	for (int v = 1; v < num_vertices; ++v)
	{
		get_neighbours<<<bpg2,tpb2>>>(v, d_neighs, d_mask, num_vertices, ful_vertices);
		cudaMemcpy(h_neighs, d_neighs, size_neighs, cudaMemcpyDeviceToHost);
		
		for (int i = 0; i < num_vertices; ++i) {
			if (h_neighs[i] == 1){
				if (color[i] != -1)
					is_neighbour_color[color[i]] = true;
			}
		}

		// Finding first unassigned colour
		int c;
		for (c = 1; c <= num_vertices; ++c) {
			if (is_neighbour_color[c] == false) break;
		}
		color[v] = c;

		for (int i = 0; i < num_vertices; ++i) {
			if (h_neighs[i] == 1){
				if (color[i] != -1)
					is_neighbour_color[color[i]] = false;
			}
		}

		if(c > num_colors_used) num_colors_used = c;
	}


	cudaEventRecord(stop);

	cudaEventSynchronize(stop);
	float milliseconds = 0;
	cudaEventElapsedTime(&milliseconds, start, stop);

	printf("number of colours used: %d\n",num_colors_used);

	printf("time taken is %.5f\n", milliseconds);

	cudaFree(d_neighs);
	cudaFree(d_hash_value);
	cudaFree(d_bits);
	cudaFree(d_mask);

	free(h_u);
	free(h_v);
	free(h_bits);
	
	cudaDeviceReset();
	return 0;
}

'File written in /content/src/vc-as.cu'

In [ ]:
!nvcc /content/src/vc-as.cu -o vc-as

/content/src/vc-as.cu(560): warning: variable "err" was declared but never referenced

/content/src/vc-as.cu(561): warning: variable "INF" was declared but never referenced



In [ ]:
!./vc-as < as733.txt

0.00779
time taken is 7.79309
value is 677252
number of colours used: 12
time taken is 569.59589


In [ ]:
!nvprof --unified-memory-profiling off ./vc-as < as733.txt

==547== NVPROF is profiling process 547, command: ./vc-as
0.00830
time taken is 8.30566
value is 677252
number of colours used: 12
time taken is 586.61633
==547== Profiling application: ./vc-as
==547== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   62.16%  167.37ms       160  1.0461ms  891.68us  1.0655ms  set_mask(bool*, unsigned long*, bool*, int, int, int, long, int)
                   31.09%  83.714ms      6473  12.932us  6.8480us  29.855us  get_neighbours(int, bool*, bool*, int, long)
                    4.01%  10.787ms      6474  1.6660us  1.5350us  108.09us  [CUDA memcpy DtoH]
                    2.70%  7.2778ms         4  1.8194ms  1.6162ms  1.9629ms  get_mask(int*, int*, bool*, int, int, long, long, int, int, int, unsigned long*)
                    0.05%  125.08us         3  41.694us  5.9200us  112.96us  [CUDA memcpy HtoD]
                    0.00%  1.2480us         1  1.2480us  1.2480us  1.2480us  [CUDA m

In [ ]:
%%cuda --name vc-ca.cu

#include <stdio.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <math.h>
#include <stdint.h>
#include <inttypes.h>

#define FORCE_INLINE

__device__ int Parent(int node);
__device__ int LeftChild(int node);
__device__ int RightChild(int node);
__device__ int Sibling(int node);
__device__ int calculate_lca(int u, int v);
__device__ void traversal(long int prev, long int lca, long int src, long int dest, int n, uint64_t *hash, bool *bit, int h, int m);
__device__ bool check_traversal_up(int prev, int lca, int src, int dest, int n, bool *mask);
__device__ bool check_traversal_down(int prev, int lca, int src, int dest, int n, bool *mask);
__device__ bool CheckBloom(long tid, uint64_t *hash_value, bool *bit, int h, int m);
__device__ void SetBloom(long tid, uint64_t *hash_value, bool *bit, int h, int m);

__device__ static inline FORCE_INLINE uint64_t rotl64 ( uint64_t x, int8_t r )
{
  return (x << r) | (x >> (64 - r));
}

#define ROTL64(x,y)	rotl64(x,y)
#define BIG_CONSTANT(x) (x##LLU)

#define getblock(p, i) (p[i])

__device__ static inline FORCE_INLINE uint64_t fmix64 ( uint64_t k )
{
  k ^= k >> 33;
  k *= BIG_CONSTANT(0xff51afd7ed558ccd);
  k ^= k >> 33;
  k *= BIG_CONSTANT(0xc4ceb9fe1a85ec53);
  k ^= k >> 33;

  return k;
}

__device__ void MurmurHash3_x64_128 ( int tid, char key[], const int len,
                           const uint32_t seed, void * out1 , void * out2)
{
  //const uint8_t * data = (const uint8_t*)key;
  //printf("tid = %d, %c, %c, %c, %c, %c, %c, %c, %c, %c\n",tid,key[0],key[1],key[2],key[3],key[4],key[5],key[6],key[7],key[8]);

  //if (key[0]== 48) {printf("h "); key[0] = 48+'\0';}

  uint8_t data[16];
  for (int i=0; i<len; i++)
    data[i] = (uint8_t) key[i];
  const int nblocks = len / 16;
  int i;

  uint64_t h1 = seed;
  uint64_t h2 = seed;

  uint64_t c1 = BIG_CONSTANT(0x87c37b91114253d5);
  uint64_t c2 = BIG_CONSTANT(0x4cf5ad432745937f);

  //----------
  // body

  //const uint64_t * blocks = (const uint64_t *)(data);

  uint64_t blocks[16];
  for (int i=0; i<len; i++)
    blocks[i] = (uint64_t) data[i];

  for(i = 0; i < nblocks; i++)
  {
    uint64_t k1 = blocks[i*2+0];
    uint64_t k2 = blocks[i*2+1];

    k1 *= c1; k1  = ROTL64(k1,31); k1 *= c2; h1 ^= k1;

    h1 = ROTL64(h1,27); h1 += h2; h1 = h1*5+0x52dce729;

    k2 *= c2; k2  = ROTL64(k2,33); k2 *= c1; h2 ^= k2;

    h2 = ROTL64(h2,31); h2 += h1; h2 = h2*5+0x38495ab5;
  }

  //----------
  // tail

  //const uint8_t * tail = (const uint8_t*)(data + nblocks*16);

  uint8_t tail[16];
  for (int i=0; i<len-nblocks*16; i++)
    tail[i] = (uint64_t) data[i+nblocks*16];

  uint64_t k1 = 0;
  uint64_t k2 = 0;

  switch(len & 15)
  {
  case 15: k2 ^= (uint64_t)(tail[14]) << 48;
  case 14: k2 ^= (uint64_t)(tail[13]) << 40;
  case 13: k2 ^= (uint64_t)(tail[12]) << 32;
  case 12: k2 ^= (uint64_t)(tail[11]) << 24;
  case 11: k2 ^= (uint64_t)(tail[10]) << 16;
  case 10: k2 ^= (uint64_t)(tail[ 9]) << 8;
  case  9: k2 ^= (uint64_t)(tail[ 8]) << 0;
           k2 *= c2; k2  = ROTL64(k2,33); k2 *= c1; h2 ^= k2;

  case  8: k1 ^= (uint64_t)(tail[ 7]) << 56;
  case  7: k1 ^= (uint64_t)(tail[ 6]) << 48;
  case  6: k1 ^= (uint64_t)(tail[ 5]) << 40;
  case  5: k1 ^= (uint64_t)(tail[ 4]) << 32;
  case  4: k1 ^= (uint64_t)(tail[ 3]) << 24;
  case  3: k1 ^= (uint64_t)(tail[ 2]) << 16;
  case  2: k1 ^= (uint64_t)(tail[ 1]) << 8;
  case  1: k1 ^= (uint64_t)(tail[ 0]) << 0;
           k1 *= c1; k1  = ROTL64(k1,31); k1 *= c2; h1 ^= k1;
  };

  //----------
  // finalization

  h1 ^= len; h2 ^= len;

  h1 += h2;
  h2 += h1;

  h1 = fmix64(h1);
  h2 = fmix64(h2);

  h1 += h2;
  h2 += h1;

  ((uint64_t*)out1)[0] = h1;
  ((uint64_t*)out2)[0] = h2;

}

__device__ inline uint64_t NthHash(uint8_t n, uint64_t hashA, uint64_t hashB, uint64_t filter_size) {
	return ((hashA + n * hashB) % filter_size);
}


__global__ void get_mask(int *u, int *v, bool *bit, int n, int e, long int ful_vertices,long int valsperloop,int ii,int h, int m, uint64_t *hash)
{
	int blockNum = blockIdx.z*(gridDim.x*gridDim.y)+blockIdx.y*gridDim.x+blockIdx.x;
	int threadNum = threadIdx.z*(blockDim.x*blockDim.y)+threadIdx.y*blockDim.x+threadIdx.x;
	int tid = blockNum*(blockDim.x*blockDim.y*blockDim.z)+threadNum;

	long int tid1 = ii*valsperloop + tid;

	if (tid1 < e){
		u[tid] = u[tid] + n - 1;
		v[tid] = v[tid] + n - 1;
		int src = u[tid];
		int dest = v[tid];

		if (!((u[tid] < ful_vertices && v[tid] < ful_vertices) || (u[tid] >= ful_vertices && v[tid] >= ful_vertices))) {
			if (u[tid] > v[tid]){
	 			int cur = Parent(u[tid]);
				SetBloom((cur*n+u[tid]-n+1) << 1,hash+2*tid*sizeof(uint64_t),bit,h,m);
				if (u[tid] == LeftChild(cur)){
					SetBloom((cur*n+v[tid]-n+1) << 1,hash+2*tid*sizeof(uint64_t),bit,h,m); 
				}
				else{
					SetBloom(((cur*n+v[tid]-n+1) << 1) + 1,hash+2*tid*sizeof(uint64_t),bit,h,m); 
				}
				u[tid] = cur;
			}	
		 	else{
	 			int cur = Parent(v[tid]);
				SetBloom((cur*n+v[tid]-n+1) << 1,hash+2*tid*sizeof(uint64_t),bit,h,m);
				if (v[tid] == LeftChild(cur)){
					SetBloom((cur*n+u[tid]-n+1) << 1,hash+2*tid*sizeof(uint64_t),bit,h,m); 
				}
				else{
					SetBloom(((cur*n+u[tid]-n+1) << 1) + 1,hash+2*tid*sizeof(uint64_t),bit,h,m); 
				}
				v[tid] = cur;
			}
		}
		
		__syncthreads();

		int lca = calculate_lca(u[tid], v[tid]);

		traversal(u[tid], lca, src, dest, n, hash+2*tid*sizeof(uint64_t),bit,h,m);
		traversal(v[tid], lca, dest, src, n, hash+2*tid*sizeof(uint64_t),bit,h,m);
	}
}

__global__ void set_mask(bool *mask, uint64_t *hash_value, bool *bit, int h, int m, int n, long int valsperloop, int ii)
{
	int blockNum = blockIdx.z*(gridDim.x*gridDim.y)+blockIdx.y*gridDim.x+blockIdx.x;
	int threadNum = threadIdx.z*(blockDim.x*blockDim.y)+threadIdx.y*blockDim.x+threadIdx.x;
	int tid = blockNum*(blockDim.x*blockDim.y*blockDim.z)+threadNum;

	if ((valsperloop*ii + tid) < n){	
		long tid1 = valsperloop*ii + tid;
		mask[tid] = CheckBloom(tid1, (hash_value)+2*tid*sizeof(uint64_t), bit, h, m);
	}
}

__global__ void check_bloom(int *found, bool *bit, bool *mask, uint64_t *hash_value, int m, int h, int n)
{
	int val = blockIdx.x;
	int hash = threadIdx.x;
	uint64_t filter_size = (uint64_t) m;
	uint8_t hash_no = (uint8_t) hash;
	
	if (val < n && *found == 0){
		if (mask[val] == 1){
			 if (hash < h){
				if (bit[NthHash(hash_no,*(hash_value+2*val*sizeof(uint64_t)),*(hash_value+(2*val+1)*sizeof(uint64_t)), filter_size)] == 0){ 		
					atomicAdd(found, 1); //printf("here");
				}
			}
		}
	}
}


__global__ void get_neighbours(int u, bool *neighs, bool *mask, int n, long int ful_vertices)
{
	int blockNum = blockIdx.z*(gridDim.x*gridDim.y)+blockIdx.y*gridDim.x+blockIdx.x;
	int threadNum = threadIdx.z*(blockDim.x*blockDim.y)+threadIdx.y*blockDim.x+threadIdx.x;
	int tid = blockNum*(blockDim.x*blockDim.y*blockDim.z)+threadNum;

	long v=tid;

	*(neighs+(tid)*sizeof(bool)) = 0;

	if (tid < n && !(u==tid) ){

		u = u + n - 1;
		v = v + n - 1;
		long src = u;
		long dest = v;

		if (!((u < ful_vertices && v < ful_vertices) || (u >= ful_vertices && v >= ful_vertices))) {
			if (u > v){
	 			long cur = Parent(u);
				if ((mask[(cur*n+u-n+1) << 1] == 0)) return;
				u = cur;
			}	
		 	else{
	 			long cur = Parent(v);
				if (v == LeftChild(cur))
				{
					if ((mask[(cur*n+u-n+1) << 1] == 0)) 
						return;
				}
				else
				{
					if ((mask[((cur*n+u-n+1) << 1) + 1] == 0)) 
						return;
				} 
				v = cur;
			}
		}
	
		//__syncthreads();

		int lca = calculate_lca(u, v);
		if (check_traversal_up(u, lca, src, dest, n, mask)) 
		{
			if(check_traversal_down(v, lca, dest, src, n, mask))
			{
				//if ((src-n+1) == 6) printf("%d ",dest-n+1);
				*(neighs+(dest-n+1)*sizeof(bool)) = 1;
			}
		}
	}
}

__device__ int Parent(int node)
{
	return (((node + 1) >> 1) - 1); 
}

__device__ int LeftChild(int node)
{
	return (((node + 1) << 1) - 1); 
}

__device__ int RightChild(int node)
{
	return ((node + 1) << 1); 
}

__device__ int Sibling(int node)
{
	return (((node + 1) ^ 1) - 1); 
}

__device__ int calculate_lca(int u, int v)
{	
	int val1 = 0;
	int val2 = 0;	
	int i = 1;

	do{
		float pow_val = 1 << i;
		val1 = floor((u+1)/pow_val);
		val2 = floor((v+1)/pow_val);
		i++;
	} while(val1 != val2);
	return (val1 - 1);
}

__device__ void traversal(long int prev, long int lca, long int src, long int dest, int n, uint64_t *hash, bool *bit, int h, int m)
{
	int cur = Parent(prev);
	while (cur != lca){
		SetBloom((cur*n+src-n+1) << 1,hash,bit,h,m);
		if (prev == LeftChild(cur)){
			SetBloom((cur*n+dest-n+1) << 1,hash,bit,h,m); 
		}
		else{
			SetBloom(((cur*n+dest-n+1) << 1) + 1,hash,bit,h,m); 
		} 
		prev = cur;
		cur = Parent(cur);
	}
	SetBloom(((cur*n+src-n+1) << 1) + 1,hash,bit,h,m);	
}

__device__ bool check_traversal_up(int prev, int lca, int src, int dest, int n, bool *mask)  
{
	int cur = Parent(prev);
	while (cur != lca){
		if ((mask[(cur*n+src-n+1) << 1]) == 0)  return false;
		prev = cur;
		cur = Parent(cur);
	}
	if ((mask[((cur*n+src-n+1) << 1) + 1]) == 0)  return false;
	return true;	
}

__device__ bool check_traversal_down(int prev, int lca, int src, int dest, int n, bool *mask) 
{
	int cur = Parent(prev);
	while (cur != lca){
		if (prev == LeftChild(cur))
		{
			if ((mask[(cur*n+dest-n+1) << 1]) == 0)  
				return false;
		}
		else
		{
			if ((mask[((cur*n+dest-n+1) << 1) + 1]) == 0) 
				return false;
		}
		prev = cur;
		cur = Parent(cur);
	}
	return true;	
}

__device__ void SetBloom(long tid, uint64_t *hash_value, bool *bit, int h, int m)
{
	int i =0;
	int count=0; 	
	long num = tid;
	char str[11];
	do{	
		count++;
		num /= 10;
	}while(num != 0); 
	num = tid;
	do{	
		str[count-i-1] = num%10 + '0';
		num/=10;
		i++;
	}while(num !=0);	
	str[i] = 48+'\0';
	uint64_t len1 = (uint64_t) count;
	size_t len = (size_t) len1;
	MurmurHash3_x64_128(tid, str, len, 0, (hash_value), (hash_value)+sizeof(uint64_t));

	for (int i=0; i<h; i++){
		bit[NthHash(i,*(hash_value),*((hash_value)+sizeof(uint64_t)),m)] = 1;
	}
}

__device__ bool CheckBloom(long tid, uint64_t *hash_value, bool *bit, int h, int m)
{
	int i =0;
	int count=0; 	
	long num = tid;
	char str[10];
	do{	
		count++;
		num /= 10;
	}while(num != 0); 
	num = tid;
	do{	
		str[count-i-1] = num%10 + '0';
		num/=10;
		i++;
	}while(num !=0);	
	str[i] = 48+'\0';
	uint64_t len1 = (uint64_t) count;
	size_t len = (size_t) len1;
	MurmurHash3_x64_128(tid, str, len, 0, (hash_value), (hash_value)+sizeof(uint64_t));

	for (int i=0; i<h; i++){
		if (bit[NthHash(i,*(hash_value),*((hash_value)+sizeof(uint64_t)),m)] == 0){
			return false;
		}
	}
	return true;
}

void InsertEdge(int num_vertices, int num_edges, int num_hashes, int num_bits, int *h_u, int *h_v, bool *h_bits)
{

	cudaError_t err = cudaSuccess;
	cudaEvent_t start1, stop1;
	cudaEventCreate(&start1);
	cudaEventCreate(&stop1);

	size_t size = num_edges * sizeof(int);

	int *d_u = NULL, *d_v = NULL; 
    err=cudaMalloc((void **)&d_u, size);
	cudaDeviceSynchronize();
	if (err != cudaSuccess)
	{
	    fprintf(stderr, "Failed to allocate vector u (error code %s)!\n", cudaGetErrorString(err));
	    exit(EXIT_FAILURE);
	}
	err=cudaMalloc((void **)&d_v, size);
	cudaDeviceSynchronize();
	if (err != cudaSuccess)
	{
	    fprintf(stderr, "Failed to allocate vector v (error code %s)!\n", cudaGetErrorString(err));
	    exit(EXIT_FAILURE);
	}
	err=cudaMemcpy(d_u, h_u, size, cudaMemcpyHostToDevice);
	cudaDeviceSynchronize();
	if (err != cudaSuccess)
	{
	    fprintf(stderr, "Failed to copy u (error code %s)!\n", cudaGetErrorString(err));
	    exit(EXIT_FAILURE);
	}
	err=cudaMemcpy(d_v, h_v, size, cudaMemcpyHostToDevice);
	cudaDeviceSynchronize();
	if (err != cudaSuccess)
	{
	    fprintf(stderr, "Failed to copy v (error code %s)!\n", cudaGetErrorString(err));
	    exit(EXIT_FAILURE);
	}

	int num_ful_levels = floor( log2((double) (2*num_vertices - 1)));
	long int ful_vertices = pow((int) 2,(int) num_ful_levels) - 1;


	size_t size_bits = num_bits * sizeof(bool);
	bool *d_bits = NULL;
    err=cudaMalloc((void **)&d_bits, size_bits);
	cudaDeviceSynchronize();
	if (err != cudaSuccess)
	{
	    fprintf(stderr, "Failed to allocate vector bits (error code %s)!\n", cudaGetErrorString(err));
	    exit(EXIT_FAILURE);
	}

	cudaMemset(d_bits, 0, size_bits);

	clock_t ti;

	cudaEventRecord(start1);
	ti=clock();

	dim3 tpb1(32,32,1);
    dim3 bpg1(2,1,1);

	long int valsperloop = 1 << 11;

	int N = ceil(num_edges/valsperloop)+1;

	uint64_t *d_hash_value = NULL;
	size_t size_hash = 2*valsperloop*sizeof(uint64_t);
	cudaMalloc((void **)&d_hash_value, size_hash);

	//printf("N is %d\n",N);

	for (int i=0; i<N; i++){
		get_mask<<<bpg1,tpb1>>>(d_u+i*valsperloop, d_v+i*valsperloop, d_bits, num_vertices, num_edges, ful_vertices, valsperloop, i, num_hashes, num_bits, d_hash_value);
	}

	cudaMemcpy(h_bits, d_bits, size_bits, cudaMemcpyDeviceToHost);

	ti=clock() -ti;
	cudaEventRecord(stop1);

	cudaEventSynchronize(stop1);
	float milliseconds = 0;
	cudaEventElapsedTime(&milliseconds, start1, stop1);

	printf("%.5f\n", float(ti) / CLOCKS_PER_SEC);

	printf("time taken is %.5f\n", milliseconds);

	int value=0;

	for (long i =0; i<num_bits; i++){
		if (h_bits[i] == 1) value++;
	}

	printf("value is %d\n",value);

	cudaFree(d_u);
	cudaFree(d_v);
	cudaFree(d_hash_value);
	cudaFree(d_bits);
}


int main ()
{

	cudaError_t err = cudaSuccess;
	const int INF = 1e8;
	int num_vertices, num_edges, num_hashes, num_bits;
	scanf("%d",&num_vertices);
	scanf("%d",&num_edges);
	scanf("%d",&num_bits);
	scanf("%d",&num_hashes);

	cudaEvent_t start, stop;
	cudaEventCreate(&start);
	cudaEventCreate(&stop);

	size_t size = num_edges * sizeof(int);
	int num_vals = 2*num_vertices*(num_vertices-1);

	int *h_u = (int *)malloc(size);
	int *h_v = (int *)malloc(size);
	
	for (int i =0; i<num_edges; i++)
	{
		scanf("%d",&h_u[i]);
		scanf("%d",&h_v[i]);
	}

	size_t size_bits = num_bits * sizeof(bool);
	bool *h_bits = (bool *)malloc(size_bits);

	InsertEdge(num_vertices, num_edges, num_hashes, num_bits, h_u, h_v, h_bits);

	size_t size_mask = num_vals * sizeof(bool);
	bool *d_mask = NULL;
    	cudaMalloc((void **)&d_mask, size_mask);

	bool *d_bits = NULL;
    	cudaMalloc((void **)&d_bits, size_bits);
	cudaMemcpy(d_bits, h_bits, size_bits, cudaMemcpyHostToDevice);

	bool *d_neighs = NULL;
	size_t size_neighs = (num_vertices)*sizeof(bool);
	cudaMalloc((void **)&d_neighs, size_neighs);
	bool *h_neighs = (bool *)malloc(size_neighs);
	
	int num_ful_levels = floor( log2((double) (2*num_vertices - 1)));
	long int ful_vertices = pow((int) 2,(int) num_ful_levels) - 1;

	size_t size_c = num_vertices * sizeof(int);
	int *color = (int *)malloc(size_c);
	size_t size_n = num_vertices * sizeof(bool);
	bool *is_neighbour_color = (bool *)malloc(size_n);

	dim3 tpb2(32,32,1);
    	dim3 bpg2(19,1,1);

	dim3 tpb3(32,32,1);
    	dim3 bpg3(4,1,1);

	long int valsperloop = 1 << 12;

	int N = ceil(num_vals/valsperloop)+1;

	uint64_t *d_hash_value = NULL;
	size_t size_hash = 2*valsperloop*sizeof(uint64_t);
	cudaMalloc((void **)&d_hash_value, size_hash);
 
	//printf("N is %d\n",N);

	clock_t ti;

	cudaEventRecord(start);

	ti=clock();

	for (int c = 1; c <= num_vertices ; ++c) {
		is_neighbour_color[c] = false;
	}

	for (int v = 0; v < num_vertices; ++v) {
		color[v] = -1;
	}

	color[0] = 1;
	int num_colors_used = 1;

	for (int i=0; i<N; i++){
		set_mask<<<bpg3,tpb3>>>(d_mask+i*valsperloop,d_hash_value,d_bits,num_hashes,num_bits,num_vals,valsperloop,i);

		/*cudaDeviceSynchronize();
	
		err = cudaGetLastError();

		if (err != cudaSuccess)
		{
		    fprintf(stderr, "Failed to launch set_mask kernel %d (error code %s)!\n",i, cudaGetErrorString(err));
		    exit(EXIT_FAILURE);
		}*/
	}

	for (int v = 1; v < num_vertices; ++v)
	{
		get_neighbours<<<bpg2,tpb2>>>(v, d_neighs, d_mask, num_vertices, ful_vertices);
		cudaMemcpy(h_neighs, d_neighs, size_neighs, cudaMemcpyDeviceToHost);
		
		for (int i = 0; i < num_vertices; ++i) {
			if (h_neighs[i] == 1){
				if (color[i] != -1)
					is_neighbour_color[color[i]] = true;
			}
		}

		// Finding first unassigned colour
		int c;
		for (c = 1; c <= num_vertices; ++c) {
			if (is_neighbour_color[c] == false) break;
		}
		color[v] = c;

		for (int i = 0; i < num_vertices; ++i) {
			if (h_neighs[i] == 1){
				if (color[i] != -1)
					is_neighbour_color[color[i]] = false;
			}
		}

		if(c > num_colors_used) num_colors_used = c;
	}


	ti=clock()-ti;

	cudaEventRecord(stop);

	cudaEventSynchronize(stop);
	float milliseconds = 0;
	cudaEventElapsedTime(&milliseconds, start, stop);
 
    printf("number of colours used: %d\n",num_colors_used);

	printf("%.5f\n", float(ti) / CLOCKS_PER_SEC);

	printf("time taken is %.5f\n", milliseconds);

	cudaFree(d_neighs);
	cudaFree(d_hash_value);
	cudaFree(d_bits);
	cudaFree(d_mask);

	free(h_u);
	free(h_v);
	free(h_bits);
	
	cudaDeviceReset();
	return 0;
}

'File written in /content/src/vc-ca.cu'

In [ ]:
!nvcc /content/src/vc-ca.cu -o vc-ca

/content/src/vc-ca.cu(524): warning: variable "err" was declared but never referenced

/content/src/vc-ca.cu(525): warning: variable "INF" was declared but never referenced



In [ ]:
!./vc-ca < CA_out.txt

0.31745
time taken is 317.54901
value is 1535956
number of colours used: 10227
5.64807
time taken is 5649.22998


In [ ]:
!nvprof --unified-memory-profiling off ./vc-ca < CA_out.txt

==603== NVPROF is profiling process 603, command: ./vc-ca
0.34639
time taken is 346.35535
value is 1535956
number of colours used: 10227
5.73698
time taken is 5736.67676
==603== Profiling application: ./vc-ca
==603== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   78.02%  2.63443s    172056  15.311us  12.063us  16.160us  set_mask(bool*, unsigned long*, bool*, int, int, int, long, int)
                   10.32%  348.36ms     18771  18.558us  12.608us  33.055us  get_neighbours(int, bool*, bool*, int, long)
                   10.21%  344.91ms       194  1.7779ms  842.95us  2.5114ms  get_mask(int*, int*, bool*, int, int, long, long, int, int, int, unsigned long*)
                    1.44%  48.579ms     18772  2.5870us  2.3030us  123.64us  [CUDA memcpy DtoH]
                    0.01%  460.59us         3  153.53us  142.88us  159.45us  [CUDA memcpy HtoD]
                    0.00%  1.6000us         1  1.6000us  1.6000us  1.

In [ ]:
%%cuda --name vc-en.cu

#include <stdio.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <math.h>
#include <stdint.h>
#include <inttypes.h>

#define FORCE_INLINE

__device__ int Parent(int node);
__device__ int LeftChild(int node);
__device__ int RightChild(int node);
__device__ int Sibling(int node);
__device__ int calculate_lca(int u, int v);
__device__ void traversal(long int prev, long int lca, long int src, long int dest, int n, uint64_t *hash, bool *bit, int h, int m);
__device__ bool check_traversal_up(int prev, int lca, int src, int dest, int n, bool *mask);
__device__ bool check_traversal_down(int prev, int lca, int src, int dest, int n, bool *mask);
__device__ bool CheckBloom(long tid, uint64_t *hash_value, bool *bit, int h, int m);
__device__ void SetBloom(long tid, uint64_t *hash_value, bool *bit, int h, int m);

__device__ static inline FORCE_INLINE uint64_t rotl64 ( uint64_t x, int8_t r )
{
  return (x << r) | (x >> (64 - r));
}

#define ROTL64(x,y)	rotl64(x,y)
#define BIG_CONSTANT(x) (x##LLU)

#define getblock(p, i) (p[i])

__device__ static inline FORCE_INLINE uint64_t fmix64 ( uint64_t k )
{
  k ^= k >> 33;
  k *= BIG_CONSTANT(0xff51afd7ed558ccd);
  k ^= k >> 33;
  k *= BIG_CONSTANT(0xc4ceb9fe1a85ec53);
  k ^= k >> 33;

  return k;
}

__device__ void MurmurHash3_x64_128 ( int tid, char key[], const int len,
                           const uint32_t seed, void * out1 , void * out2)
{
  //const uint8_t * data = (const uint8_t*)key;
  //printf("tid = %d, %c, %c, %c, %c, %c, %c, %c, %c, %c, %c, %c\n",tid,key[0],key[1],key[2],key[3],key[4],key[5],key[6],key[7],key[8],key[9],key[10]);

  //if (key[0]== 48) {printf("h "); key[0] = 48+'\0';}

  uint8_t data[16];
  for (int i=0; i<len; i++)
    data[i] = (uint8_t) key[i];
  const int nblocks = len / 16;
  int i;

  uint64_t h1 = seed;
  uint64_t h2 = seed;

  uint64_t c1 = BIG_CONSTANT(0x87c37b91114253d5);
  uint64_t c2 = BIG_CONSTANT(0x4cf5ad432745937f);

  //----------
  // body

  //const uint64_t * blocks = (const uint64_t *)(data);

  uint64_t blocks[16];
  for (int i=0; i<len; i++)
    blocks[i] = (uint64_t) data[i];

  for(i = 0; i < nblocks; i++)
  {
    uint64_t k1 = blocks[i*2+0];
    uint64_t k2 = blocks[i*2+1];

    k1 *= c1; k1  = ROTL64(k1,31); k1 *= c2; h1 ^= k1;

    h1 = ROTL64(h1,27); h1 += h2; h1 = h1*5+0x52dce729;

    k2 *= c2; k2  = ROTL64(k2,33); k2 *= c1; h2 ^= k2;

    h2 = ROTL64(h2,31); h2 += h1; h2 = h2*5+0x38495ab5;
  }

  //----------
  // tail

  //const uint8_t * tail = (const uint8_t*)(data + nblocks*16);

  uint8_t tail[16];
  for (int i=0; i<len-nblocks*16; i++)
    tail[i] = (uint64_t) data[i+nblocks*16];

  uint64_t k1 = 0;
  uint64_t k2 = 0;

  switch(len & 15)
  {
  case 15: k2 ^= (uint64_t)(tail[14]) << 48;
  case 14: k2 ^= (uint64_t)(tail[13]) << 40;
  case 13: k2 ^= (uint64_t)(tail[12]) << 32;
  case 12: k2 ^= (uint64_t)(tail[11]) << 24;
  case 11: k2 ^= (uint64_t)(tail[10]) << 16;
  case 10: k2 ^= (uint64_t)(tail[ 9]) << 8;
  case  9: k2 ^= (uint64_t)(tail[ 8]) << 0;
           k2 *= c2; k2  = ROTL64(k2,33); k2 *= c1; h2 ^= k2;

  case  8: k1 ^= (uint64_t)(tail[ 7]) << 56;
  case  7: k1 ^= (uint64_t)(tail[ 6]) << 48;
  case  6: k1 ^= (uint64_t)(tail[ 5]) << 40;
  case  5: k1 ^= (uint64_t)(tail[ 4]) << 32;
  case  4: k1 ^= (uint64_t)(tail[ 3]) << 24;
  case  3: k1 ^= (uint64_t)(tail[ 2]) << 16;
  case  2: k1 ^= (uint64_t)(tail[ 1]) << 8;
  case  1: k1 ^= (uint64_t)(tail[ 0]) << 0;
           k1 *= c1; k1  = ROTL64(k1,31); k1 *= c2; h1 ^= k1;
  };

  //----------
  // finalization

  h1 ^= len; h2 ^= len;

  h1 += h2;
  h2 += h1;

  h1 = fmix64(h1);
  h2 = fmix64(h2);

  h1 += h2;
  h2 += h1;

  ((uint64_t*)out1)[0] = h1;
  ((uint64_t*)out2)[0] = h2;

}

__device__ inline uint64_t NthHash(uint8_t n, uint64_t hashA, uint64_t hashB, uint64_t filter_size) {
	//printf("%" PRIu64 " and %" PRIu64 " %u \n",hashA,hashB,((hashA + n * hashB) % filter_size));
	//printf("%u\n",n);
	//printf ("nth %u\n",((hashA + n * hashB) % filter_size));
	return ((hashA + n * hashB) % filter_size);
}


__global__ void get_mask(int *u, int *v, bool *bit, int n, int e, long int ful_vertices,long int valsperloop,int ii,int h, int m, uint64_t *hash)
{
	int blockNum = blockIdx.z*(gridDim.x*gridDim.y)+blockIdx.y*gridDim.x+blockIdx.x;
	int threadNum = threadIdx.z*(blockDim.x*blockDim.y)+threadIdx.y*blockDim.x+threadIdx.x;
	int tid = blockNum*(blockDim.x*blockDim.y*blockDim.z)+threadNum;

	long int tid1 = ii*valsperloop + tid;

	//if (ii == 1) printf("%d",tid);

	if (tid1 < e){

		u[tid] = u[tid] + n - 1;
		v[tid] = v[tid] + n - 1;
		int src = u[tid];
		int dest = v[tid];

		if (!((u[tid] < ful_vertices && v[tid] < ful_vertices) || (u[tid] >= ful_vertices && v[tid] >= ful_vertices))) {
			if (u[tid] > v[tid]){
	 			int cur = Parent(u[tid]);
				SetBloom((cur*n+u[tid]-n+1) << 1,hash+2*tid*sizeof(uint64_t),bit,h,m);
				if (u[tid] == LeftChild(cur)){
					SetBloom((cur*n+v[tid]-n+1) << 1,hash+2*tid*sizeof(uint64_t),bit,h,m); 
				}
				else{
					SetBloom(((cur*n+v[tid]-n+1) << 1) + 1,hash+2*tid*sizeof(uint64_t),bit,h,m); 
				}
				u[tid] = cur;
			}	
		 	else{
	 			int cur = Parent(v[tid]);
				SetBloom((cur*n+v[tid]-n+1) << 1,hash+2*tid*sizeof(uint64_t),bit,h,m);
				if (v[tid] == LeftChild(cur)){
					SetBloom((cur*n+u[tid]-n+1) << 1,hash+2*tid*sizeof(uint64_t),bit,h,m); 
				}
				else{
					SetBloom(((cur*n+u[tid]-n+1) << 1) + 1,hash+2*tid*sizeof(uint64_t),bit,h,m); 
				}
				v[tid] = cur;
			}
		}
		
		//__syncthreads();

		int lca = calculate_lca(u[tid], v[tid]);

		traversal(u[tid], lca, src, dest, n, hash+2*tid*sizeof(uint64_t),bit,h,m);
		traversal(v[tid], lca, dest, src, n, hash+2*tid*sizeof(uint64_t),bit,h,m);
		
	}
}



__global__ void set_mask(bool *mask, uint64_t *hash_value, bool *bit, int h, int m, long int n, long int valsperloop, int ii)
{
	int blockNum = blockIdx.z*(gridDim.x*gridDim.y)+blockIdx.y*gridDim.x+blockIdx.x;
	int threadNum = threadIdx.z*(blockDim.x*blockDim.y)+threadIdx.y*blockDim.x+threadIdx.x;
	int tid = blockNum*(blockDim.x*blockDim.y*blockDim.z)+threadNum;

	//printf("%ld\n",valsperloop*ii);
	if ((( valsperloop*ii) + tid) < n){	
		long tid1 = valsperloop*ii + tid;
		mask[tid] = CheckBloom(tid1, (hash_value)+2*tid*sizeof(uint64_t), bit, h, m);
		//if (mask[tid] == 1) printf("its 1\n");
	}
}

__global__ void check_bloom(int *found, bool *bit, bool *mask, uint64_t *hash_value, int m, int h, int n)
{
	int val = blockIdx.x;
	int hash = threadIdx.x;
	uint64_t filter_size = (uint64_t) m;
	uint8_t hash_no = (uint8_t) hash;
	
	if (val < n && *found == 0){
		if (mask[val] == 1){
			 if (hash < h){
				if (bit[NthHash(hash_no,*(hash_value+2*val*sizeof(uint64_t)),*(hash_value+(2*val+1)*sizeof(uint64_t)), filter_size)] == 0){ 		
					atomicAdd(found, 1); //printf("here");
				}
			}
		}
	}
}


__global__ void get_neighbours(int u, bool *neighs, bool *mask, int n, long int ful_vertices)
{
	int blockNum = blockIdx.z*(gridDim.x*gridDim.y)+blockIdx.y*gridDim.x+blockIdx.x;
	int threadNum = threadIdx.z*(blockDim.x*blockDim.y)+threadIdx.y*blockDim.x+threadIdx.x;
	int tid = blockNum*(blockDim.x*blockDim.y*blockDim.z)+threadNum;

	long v=tid;

	*(neighs+(tid)*sizeof(bool)) = 0;

	if (tid < n && !(u==tid) ){

		u = u + n - 1;
		v = v + n - 1;
		long src = u;
		long dest = v;

		if (!((u < ful_vertices && v < ful_vertices) || (u >= ful_vertices && v >= ful_vertices))) {
			if (u > v){
	 			long cur = Parent(u);
				if ((mask[((long)cur*n+u-n+1) << 1] == 0)) return;
				u = cur;
			}	
		 	else{
	 			long cur = Parent(v);
				if (v == LeftChild(cur))
				{
					if ((mask[((long)cur*n+u-n+1) << 1] == 0)) 
						return;
				}
				else
				{
					if ((mask[(((long)cur*n+u-n+1) << 1) + 1] == 0)) 
						return;
				} 
				v = cur;
			}
		}
	
		//__syncthreads();

		int lca = calculate_lca(u, v);
		if (check_traversal_up(u, lca, src, dest, n, mask)) 
		{
			if(check_traversal_down(v, lca, dest, src, n, mask))
			{
				*(neighs+(dest-n+1)*sizeof(bool)) = 1;
			}
		}
	}
}

__global__ void print_hash(uint64_t *hash_value, int n)
{
	int blockNum = blockIdx.z*(gridDim.x*gridDim.y)+blockIdx.y*gridDim.x+blockIdx.x;
	int threadNum = threadIdx.z*(blockDim.x*blockDim.y)+threadIdx.y*blockDim.x+threadIdx.x;
	int tid = blockNum*(blockDim.x*blockDim.y*blockDim.z)+threadNum;

	if (tid < 4*n*(n-1)){
		//printf("%u\n",*(hash_value+tid*sizeof(uint8_t)));
		printf("%" PRIu64 " %d\n",*(hash_value+tid*sizeof(uint64_t)),tid);
	}	
}

__device__ int Parent(int node)
{
	return (((node + 1) >> 1) - 1); 
}

__device__ int LeftChild(int node)
{
	return (((node + 1) << 1) - 1); 
}

__device__ int RightChild(int node)
{
	return ((node + 1) << 1); 
}

__device__ int Sibling(int node)
{
	return (((node + 1) ^ 1) - 1); 
}

__device__ int calculate_lca(int u, int v)
{	
	int val1 = 0;
	int val2 = 0;	
	int i = 1;

	do{
		float pow_val = 1 << i;
		val1 = floor((u+1)/pow_val);
		val2 = floor((v+1)/pow_val);
		i++;
	} while(val1 != val2);
	return (val1 - 1);
}

__device__ void traversal(long int prev, long int lca, long int src, long int dest, int n, uint64_t *hash, bool *bit, int h, int m)
{
	int cur = Parent(prev);
	while (cur != lca){
		SetBloom((cur*n+src-n+1) << 1,hash,bit,h,m);
		if (prev == LeftChild(cur)){
			SetBloom((cur*n+dest-n+1) << 1,hash,bit,h,m); 
		}
		else{
			SetBloom(((cur*n+dest-n+1) << 1) + 1,hash,bit,h,m); 
		} 
		prev = cur;
		cur = Parent(cur);
	}
	SetBloom(((cur*n+src-n+1) << 1) + 1,hash,bit,h,m);	
}

__device__ bool check_traversal_up(int prev, int lca, int src, int dest, int n, bool *mask)  
{
	int cur = Parent(prev);
	while (cur != lca){
		if ((mask[((long)cur*n+src-n+1) << 1]) == 0)  return false;
		prev = cur;
		cur = Parent(cur);
	}
	if ((mask[(((long)cur*n+src-n+1) << 1) + 1]) == 0)  return false;
	return true;	
}

__device__ bool check_traversal_down(int prev, int lca, int src, int dest, int n, bool *mask) 
{
	int cur = Parent(prev);
	while (cur != lca){
		if (prev == LeftChild(cur))
		{
			if ((mask[((long)cur*n+dest-n+1) << 1]) == 0)  
				return false;
		}
		else
		{
			if ((mask[(((long)cur*n+dest-n+1) << 1) + 1]) == 0) 
				return false;
		}
		prev = cur;
		cur = Parent(cur);
	}
	return true;	
}

__device__ void SetBloom(long tid, uint64_t *hash_value, bool *bit, int h, int m)
{
	int i =0;
	int count=0; 	
	long num = tid;
	char str[11];
	do{	
		count++;
		num /= 10;
	}while(num != 0); 
	num = tid;
	do{	
		str[count-i-1] = num%10 + '0';
		num/=10;
		i++;
	}while(num !=0);	
	str[i] = 48+'\0';
	uint64_t len1 = (uint64_t) count;
	size_t len = (size_t) len1;
	MurmurHash3_x64_128(tid, str, len, 0, (hash_value), (hash_value)+sizeof(uint64_t));

	for (int i=0; i<h; i++){
		bit[NthHash(i,*(hash_value),*((hash_value)+sizeof(uint64_t)),m)] = 1;
	}
}

__device__ bool CheckBloom(long tid, uint64_t *hash_value, bool *bit, int h, int m)
{
	int i =0;
	int count=0; 	
	long num = tid;
	char str[10];
	do{	
		count++;
		num /= 10;
	}while(num != 0); 
	num = tid;
	do{	
		str[count-i-1] = num%10 + '0';
		num/=10;
		i++;
	}while(num !=0);	
	str[i] = 48+'\0';
	uint64_t len1 = (uint64_t) count;
	size_t len = (size_t) len1;
	MurmurHash3_x64_128(tid, str, len, 0, (hash_value), (hash_value)+sizeof(uint64_t));

	//("hereee\n");
	for (int i=0; i<h; i++){
		if (bit[NthHash(i,*(hash_value),*((hash_value)+sizeof(uint64_t)),m)] == 0){
			return false;
		}
	}
	return true;
}

void InsertEdge(int num_vertices, int num_edges, int num_hashes, int num_bits, int *h_u, int *h_v, bool *h_bits)
{

	//cudaError_t err = cudaSuccess;
	cudaEvent_t start1, stop1;
	cudaEventCreate(&start1);
	cudaEventCreate(&stop1);

	size_t size = num_edges * sizeof(int);

	printf("size is %zu\n",size);

	int num_ful_levels = floor( log2((double) (2*num_vertices - 1)));
	long int ful_vertices = pow((int) 2,(int) num_ful_levels) - 1;

	size_t size_bits = num_bits * sizeof(bool);
	bool *d_bits = NULL;
    cudaMalloc((void **)&d_bits, size_bits);
	cudaMemset(d_bits, 0, size_bits);

	int *d_u = NULL, *d_v = NULL; 
	cudaMalloc((void **)&d_u, size);
	cudaMalloc((void **)&d_v, size);
	cudaDeviceSynchronize();
	cudaMemcpy(d_u, h_u, size, cudaMemcpyHostToDevice);
	cudaMemcpy(d_v, h_v, size, cudaMemcpyHostToDevice);

	clock_t ti;

	cudaEventRecord(start1);
	ti=clock();

	dim3 tpb1(32,16,1);
    dim3 bpg1(4,1,1);

	long int valsperloop = 1 << 11;

	int N = ceil(num_edges/valsperloop)+1;

	uint64_t *d_hash_value = NULL;
	size_t size_hash = 2*valsperloop*sizeof(uint64_t);
	cudaMalloc((void **)&d_hash_value, size_hash);

	//printf("N is %d\n",N);

	for (int i=0; i<N; i++){
		get_mask<<<bpg1,tpb1>>>(d_u+i*valsperloop, d_v+i*valsperloop, d_bits, num_vertices, num_edges, ful_vertices, valsperloop, i, num_hashes, num_bits, d_hash_value);
	}

	cudaMemcpy(h_bits, d_bits, size_bits, cudaMemcpyDeviceToHost);

	ti=clock() -ti;
	cudaEventRecord(stop1);

	cudaEventSynchronize(stop1);
	float milliseconds = 0;
	cudaEventElapsedTime(&milliseconds, start1, stop1);

	printf("%.5f\n", float(ti) / CLOCKS_PER_SEC);

	printf("time taken is %.5f\n", milliseconds);

	int value=0;

	for (long i =0; i<num_bits; i++){
		if (h_bits[i] == 1) value++;
	}

	printf("value is %d\n",value);

	cudaFree(d_u);
	cudaFree(d_v);
	cudaFree(d_hash_value);
	cudaFree(d_bits);
}


int main ()
{

	//cudaError_t err = cudaSuccess;
	const int INF = 1e8;
	int num_vertices, num_edges, num_hashes, num_bits;
	scanf("%d",&num_vertices);
	scanf("%d",&num_edges);
	scanf("%d",&num_bits);
	scanf("%d",&num_hashes);

	cudaEvent_t start, stop;
	cudaEventCreate(&start);
	cudaEventCreate(&stop);

	size_t size = num_edges * sizeof(int);
	//int num_vals = 2*num_vertices*(num_vertices-1);
	long int num_vals = 2692532344;

	int *h_u = (int *)malloc(size);
	int *h_v = (int *)malloc(size);
	
	for (int i =0; i<num_edges; i++)
	{
		scanf("%d",&h_u[i]);
		scanf("%d",&h_v[i]);
	}

	size_t size_bits = num_bits * sizeof(bool);
	bool *h_bits = (bool *)malloc(size_bits);

	InsertEdge(num_vertices, num_edges, num_hashes, num_bits, h_u, h_v, h_bits);

	cudaDeviceSynchronize();

	//size_t size_mask = num_vals * sizeof(bool);
	size_t size_mask = 2692532344;
	bool *h_mask = (bool *)malloc(size_mask);
	bool *d_mask = NULL;
    	cudaMalloc((void **)&d_mask, size_mask);

	bool *d_bits = NULL;
    	cudaMalloc((void **)&d_bits, size_bits);
	cudaMemcpy(d_bits, h_bits, size_bits, cudaMemcpyHostToDevice);

	bool *d_neighs = NULL;
	size_t size_neighs = (num_vertices)*sizeof(bool);
	cudaMalloc((void **)&d_neighs, size_neighs);
	bool *h_neighs = (bool *)malloc(size_neighs);
	
	int num_ful_levels = floor( log2((double) (2*num_vertices - 1)));
	long int ful_vertices = pow((int) 2,(int) num_ful_levels) - 1;
	
	size_t size_c = num_vertices * sizeof(int);
	int *color = (int *)malloc(size_c);
	size_t size_n = num_vertices * sizeof(bool);
	bool *is_neighbour_color = (bool *)malloc(size_n);

	dim3 tpb2(32,32,1);
    dim3 bpg2(16,4,1);

	dim3 tpb3(32,32,1);
    dim3 bpg3(2,2,1);

	long int valsperloop = 1 << 12;

	int N = ceil(num_vals/valsperloop)+1;

	uint64_t *d_hash_value = NULL;
	size_t size_hash = 2*valsperloop*sizeof(uint64_t);
	cudaMalloc((void **)&d_hash_value, size_hash);

	//printf("size is %zu\n",size_mask);

	//printf("N is %d\n",N);

	clock_t ti;

	cudaEventRecord(start);

	ti=clock();

	for (int c = 1; c <= num_vertices ; ++c) {
		is_neighbour_color[c] = false;
	}

	for (int v = 0; v < num_vertices; ++v) {
		color[v] = -1;
	}

	color[0] = 1;
	int num_colors_used = 1;

	for (int i=0; i<N; i++){
		set_mask<<<bpg3,tpb3>>>(d_mask+i*valsperloop,d_hash_value,d_bits,num_hashes,num_bits,num_vals,valsperloop,i);
		/*cudaDeviceSynchronize();
	
		err = cudaGetLastError();

		if (err != cudaSuccess)
		{
		    fprintf(stderr, "Failed to launch set_mask kernel %d (error code %s)!\n",i, cudaGetErrorString(err));
		    exit(EXIT_FAILURE);
		}*/
	}
	
	for (int v = 1; v < num_vertices; ++v)
	{
		get_neighbours<<<bpg2,tpb2>>>(v, d_neighs, d_mask, num_vertices, ful_vertices);
		cudaMemcpy(h_neighs, d_neighs, size_neighs, cudaMemcpyDeviceToHost);
		
		for (int i = 0; i < num_vertices; ++i) {
			if (h_neighs[i] == 1){
				if (color[i] != -1)
					is_neighbour_color[color[i]] = true;
			}
		}

		// Finding first unassigned colour
		int c;
		for (c = 1; c <= num_vertices; ++c) {
			if (is_neighbour_color[c] == false) break;
		}
		color[v] = c;

		for (int i = 0; i < num_vertices; ++i) {
			if (h_neighs[i] == 1){
				if (color[i] != -1)
					is_neighbour_color[color[i]] = false;
			}
		}

		if(c > num_colors_used) num_colors_used = c;
	}


	ti=clock()-ti;

	cudaEventRecord(stop);

	cudaEventSynchronize(stop);
	float milliseconds = 0;
	cudaEventElapsedTime(&milliseconds, start, stop);

	printf("%.5f\n", float(ti) / CLOCKS_PER_SEC);
	printf("number of colours used: %d\n",num_colors_used);

	printf("time taken is %.5f\n", milliseconds);

	cudaFree(d_neighs);
	cudaFree(d_hash_value);
	cudaFree(d_bits);
	cudaFree(d_mask);

	free(h_u);
	free(h_v);
	free(h_bits);
	
	cudaDeviceReset();
	return 0;
}

'File written in /content/src/vc-en.cu'

In [ ]:
!nvcc /content/src/vc-en.cu -o vc-en

/content/src/vc-en.cu(519): warning: variable "INF" was declared but never referenced



In [ ]:
!./vc-en < EN_out.txt

size is 735324
0.12073
time taken is 121.02454
value is 1535421
tcmalloc: large alloc 2692538368 bytes == 0x56449088c000 @  0x7f4e99d261e7 0x56448d37a427 0x7f4e989b9b97 0x56448d379b2a
22.12794
number of colours used: 12247
time taken is 22131.09180


In [ ]:
!nvprof --unified-memory-profiling off ./vc-en < EN_out.txt

==663== NVPROF is profiling process 663, command: ./vc-en
size is 735324
0.10167
time taken is 101.92608
value is 1535421
tcmalloc: large alloc 2692538368 bytes == 0x55ff0f7f6000 @  0x7fc733d601e7 0x55ff0845f427 0x7fc7329f3b97 0x55ff0845eb2a
30.22527
number of colours used: 12247
time taken is 23187.70312
==663== Profiling application: ./vc-en
==663== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   86.17%  10.5392s    657357  16.032us  13.952us  25.888us  set_mask(bool*, unsigned long*, bool*, int, int, long, long, int)
                   11.69%  1.42938s     36691  38.957us  9.0240us  48.032us  get_neighbours(int, bool*, bool*, int, long)
                    1.32%  161.65ms     36692  4.4050us  3.6800us  120.96us  [CUDA memcpy DtoH]
                    0.82%  100.62ms        90  1.1180ms  635.08us  1.2357ms  get_mask(int*, int*, bool*, int, int, long, long, int, int, int, unsigned long*)
                    0.00%  

In [ ]:
!nvidia-smi

Thu Oct 22 10:51:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P8    21W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------